In [1]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:

import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
class PVNet2():
    def __init__(self, board_height, board_width):
        
        self.board_width = board_width
        self.board_height = board_height

        # Define the tensorflow neural network
        # 1. Input:
        tf.reset_default_graph()
        self.input_states = tf.placeholder(
                tf.float32, shape=[None, 4, board_height, board_width])
        self.input_state = tf.transpose(self.input_states, [0, 2, 3, 1])
        # 2. Common Networks Layers
        self.conv1 = tf.layers.conv2d(inputs=self.input_state,
                                      filters=32, kernel_size=[3, 3],
                                      padding="same", data_format="channels_last",
                                      activation=tf.nn.relu)
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
                                      kernel_size=[3, 3], padding="same",
                                      data_format="channels_last",
                                      activation=tf.nn.relu)
        self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
                                      kernel_size=[3, 3], padding="same",
                                      data_format="channels_last",
                                      activation=tf.nn.relu)
        # 3-1 Action Networks
        self.action_conv = tf.layers.conv2d(inputs=self.conv3, filters=4,
                                            kernel_size=[1, 1], padding="same",
                                            data_format="channels_last",
                                            activation=tf.nn.relu)
        # Flatten the tensor
        self.action_conv_flat = tf.reshape(
                self.action_conv, [-1, 4 * board_height * board_width])
        # 3-2 Full connected layer, the output is the log probability of moves
        # on each slot on the board
        self.action_fc = tf.layers.dense(inputs=self.action_conv_flat,
                                         units=board_height * board_width,
                                         activation=tf.nn.log_softmax)
        # 4 Evaluation Networks
        self.evaluation_conv = tf.layers.conv2d(inputs=self.conv3, filters=2,
                                                kernel_size=[1, 1],
                                                padding="same",
                                                data_format="channels_last",
                                                activation=tf.nn.relu)
        self.evaluation_conv_flat = tf.reshape(
                self.evaluation_conv, [-1, 2 * board_height * board_width])
        self.evaluation_fc1 = tf.layers.dense(inputs=self.evaluation_conv_flat,
                                              units=64, activation=tf.nn.relu)
        # output the score of evaluation on current state
        self.evaluation_fc2 = tf.layers.dense(inputs=self.evaluation_fc1,
                                              units=1, activation=tf.nn.tanh)

        # Define the Loss function
        # 1. Label: the array containing if the game wins or not for each state
        self.labels = tf.placeholder(tf.float32, shape=[None, 1])
        # 2. Predictions: the array containing the evaluation score of each state
        # which is self.evaluation_fc2
        # 3-1. Value Loss function
        self.value_loss = tf.losses.mean_squared_error(self.labels,
                                                       self.evaluation_fc2)
        # 3-2. Policy Loss function
        self.mcts_probs = tf.placeholder(
                tf.float32, shape=[None, board_height * board_width])
        self.policy_loss = tf.negative(tf.reduce_mean(
                tf.reduce_sum(tf.multiply(self.mcts_probs, self.action_fc), 1)))
        # 3-3. L2 penalty (regularization)
        l2_penalty_beta = 1e-4
        vars = tf.trainable_variables()
        l2_penalty = l2_penalty_beta * tf.add_n(
            [tf.nn.l2_loss(v) for v in vars if 'bias' not in v.name.lower()])
        # 3-4 Add up to be the Loss function
        self.loss = self.value_loss + self.policy_loss + l2_penalty

        # Define the optimizer we use for training
        self.learning_rate = tf.placeholder(tf.float32)
        self.optimizer = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)

        # Make a session
        self.session = tf.Session()

        # calc policy entropy, for monitoring only
        self.entropy = tf.negative(tf.reduce_mean(
                tf.reduce_sum(tf.exp(self.action_fc) * self.action_fc, 1)))

        # Initialize variables
        init = tf.global_variables_initializer()
        self.session.run(init)
        self.saver = tf.train.Saver()
        '''
        
        self.board_width = board_width
        self.board_height = board_height

        tf.reset_default_graph()
        # Define the tensorflow neural network
        # 1. Input:
        #tf.reset_default_graph()
        self.input_states = tf.placeholder(
                tf.float32, shape=[None, 4, board_height, board_width])
        self.input_state = tf.transpose(self.input_states, [0, 2, 3, 1])
        
        # Conv layer followed by relu activation and batchnormalization
        self.x = tf.layers.conv2d(inputs=self.input_state, filters=64, kernel_size=[3, 3],padding="same", activation=tf.nn.relu)
        self.x = tf.layers.batch_normalization(inputs=self.x)
        self.x = tf.layers.conv2d(inputs=self.x, filters=128,kernel_size=[3, 3], padding="same",activation=tf.nn.relu)
        self.x = tf.layers.batch_normalization(inputs=self.x)
        self.x = tf.layers.conv2d(inputs=self.x, filters=256,kernel_size=[3, 3], padding="same",activation=tf.nn.relu)
        self.x = tf.layers.batch_normalization(inputs=self.x)
        #print(self.x.shape)
        # Policy Net
        self.p = tf.layers.conv2d(inputs=self.x, filters=2,kernel_size=[1, 1], padding="same",activation=tf.nn.relu)
        self.p = tf.layers.batch_normalization(inputs=self.p)
        #print(self.p.shape)
        self.p = tf.reshape(self.p, [-1, 2 * board_height * board_width])
        #print(self.p.shape)
        self.action_fc = tf.layers.dense(inputs=self.p, units=board_height * board_width, activation=tf.nn.log_softmax)
        
        # Value net
        self.v = tf.layers.conv2d(inputs=self.x, filters=1, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)
        self.v = tf.layers.batch_normalization(inputs=self.v)
        self.v = tf.reshape(self.v, [-1,  board_height * board_width])
        self.v = tf.layers.dense(inputs=self.v, units=64, activation=tf.nn.relu)
        self.evaluation_fc = tf.layers.dense(inputs=self.v, units=1, activation=tf.nn.tanh)

        
        # Define the Loss function
        # 1. Label: the array containing if the game wins or not for each state
        self.labels = tf.placeholder(tf.float32, shape=[None, 1])
        # 2. Predictions: the array containing the evaluation score of each state
        # which is self.evaluation_fc2
        # 3-1. Value Loss function
        self.value_loss = tf.losses.mean_squared_error(self.labels,
                                                       self.evaluation_fc)
        # 3-2. Policy Loss function
        self.mcts_probs = tf.placeholder(
                tf.float32, shape=[None, board_height * board_width])
        self.policy_loss = tf.negative(tf.reduce_mean(
                tf.reduce_sum(tf.multiply(self.mcts_probs, self.action_fc), 1)))
        # 3-3. L2 penalty (regularization)
        l2_penalty_beta = 1e-4
        vars = tf.trainable_variables()
        l2_penalty = l2_penalty_beta * tf.add_n(
            [tf.nn.l2_loss(v) for v in vars if 'bias' not in v.name.lower()])
        # 3-4 Add up to be the Loss function
        self.loss = self.value_loss + self.policy_loss + l2_penalty

        # Define the optimizer we use for training
        self.learning_rate = tf.placeholder(tf.float32)
        self.optimizer = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)

        # Make a session
        self.session = tf.Session()

        # calc policy entropy, for monitoring only
        self.entropy = tf.negative(tf.reduce_mean(
                tf.reduce_sum(tf.exp(self.action_fc) * self.action_fc, 1)))

        # Initialize variables
        init = tf.global_variables_initializer()
        self.session.run(init)

        # For saving and restoring
        self.saver = tf.train.Saver()
        '''
    def predict(self, state, v=True):
        #state.get_state().reshape(-1, 1, 8, 8)
        log_act_probs, value = self.session.run(
            [self.action_fc, self.evaluation_fc2],
            feed_dict={self.input_states: state}
            )
        act_probs = np.exp(log_act_probs)
        #print("predict", act_probs, value[0][0])
        if v:
            return act_probs[0], value[0][0]
        else:
            return act_probs, value
       
    
    def train(self, state_batch, mcts_probs, winner_batch, lr):
        winner_batch = np.reshape(winner_batch, (-1, 1))
        loss, entropy, _ = self.session.run(
                [self.loss, self.entropy, self.optimizer],
                feed_dict={self.input_states: state_batch,
                           self.mcts_probs: mcts_probs,
                           self.labels: winner_batch,
                           self.learning_rate: lr})
        return loss, entropy
    
    def save(self, path):
        self.saver.save(self.session, path)
        
    def restore(self, path):
        self.saver.restore(self.session, path)
        

In [3]:
from keras.regularizers import l2
class PVNet():
    '''Policy Value Neural Network'''
    def __init__(self, dimension):
        self.dimension = dimension
        self.l2_const = 1e-4
        self.initialize()
        
        
    def initialize2(self):
        in_x = network = Input((1, 8, 8))

        # conv layers
        network = Conv2D(filters=32, kernel_size=(3, 3), padding="same", data_format="channels_first", activation="relu", kernel_regularizer=l2(self.l2_const))(network)
        network = Conv2D(filters=64, kernel_size=(3, 3), padding="same", data_format="channels_first", activation="relu", kernel_regularizer=l2(self.l2_const))(network)
        network = Conv2D(filters=128, kernel_size=(3, 3), padding="same", data_format="channels_first", activation="relu", kernel_regularizer=l2(self.l2_const))(network)
        # action policy layers
        policy_net = Conv2D(filters=4, kernel_size=(1, 1), data_format="channels_first", activation="relu", kernel_regularizer=l2(self.l2_const))(network)
        policy_net = Flatten()(policy_net)
        self.policy_net = Dense(64, activation="softmax", kernel_regularizer=l2(self.l2_const))(policy_net)
        # state value layers
        value_net = Conv2D(filters=2, kernel_size=(1, 1), data_format="channels_first", activation="relu", kernel_regularizer=l2(self.l2_const))(network)
        value_net = Flatten()(value_net)
        value_net = Dense(64, kernel_regularizer=l2(self.l2_const))(value_net)
        self.value_net = Dense(1, activation="tanh", kernel_regularizer=l2(self.l2_const))(value_net)

        self.model = Model(in_x, [self.policy_net, self.value_net])
        
    def initialize(self):
        # Conv layers
        weight_decay=1e-4
        input = Input((1, self.dimension, self.dimension))
        x = input
        x = Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        #print(x.shape)
        
        # policy net
        p = Conv2D(2, (1,1), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        p = BatchNormalization()(p)
        p = Flatten()(p)
        p = Dense(self.dimension * self.dimension, activation='softmax')(p)
        #print(p.shape)
        
        # value net
        v = Conv2D(2, (1,1), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        v = BatchNormalization()(v)
        v = Flatten()(v)
        v = Dense(self.dimension * self.dimension, activation='relu')(v)
        v = Dense(self.dimension * self.dimension, activation='tanh')(v)
        
        self.model = Model(inputs=input, outputs=[p, v])
        self.model.compile(optimizer=Adam(), loss=['categorical_crossentropy', 'mean_squared_error'])
        
    def train(self, x, y):
        self.model.fit(x, y)
        
    def predict(self, state):
        probs, value = self.model.predict_on_batch(state)
        return probs[0], value[0][0]
    def predict2(self, state):
        probs, value = self.model.predict_on_batch(state)
        return probs[0], value[0][0]
        
        
        

In [4]:
import random
from collections import defaultdict, deque

class Node():
    def __init__(self, parent, p):
        self.parent = parent
        self.children = {}
        self.Q = 0
        self.c = 5
        self.visits = 0
        self.P = p
    
    # get the value Q + U
    def get_value(self):
        return self.Q + self.c * self.P * np.sqrt(self.parent.visits) / (1 + self.visits)
    
    # expand the leaf node 
    def expand(self, actions, probs):
        #print(actions, probs)
        for i in range(len(actions)):
            if actions[i] not in self.children:
                self.children[actions[i]] = Node(self, probs[i])
          
    # back propagate the value from leaf to root
    def update(self, value):
        self.visits += 1
        self.Q += 1.0 * (value - self.Q) / self.visits
        if self.parent:
            self.parent.update(-value)


def softmax(x):
    probs = np.exp(x - np.max(x))
    probs /= np.sum(probs)
    return probs
            
import copy, time
# Monte Carlo Tree Search
class MCTS():
    def __init__(self, pv_net, c, n_simulations):
        self.root = Node(None, 1.0)
     
        self.pv_net = pv_net
        self.c = c
        self.n_simulations = n_simulations
    
    # run n simulations of mcts
    def run(self, board, temp=1):
        #visits = [self.root.children[c].visits for c in self.root.children]
        #print("visits b", visits)
        for i in range(self.n_simulations):
            #if i % 100 == 0:
               # print(i, " simulations")
            self.traverse(self.root, copy.deepcopy(board))
            
        #print("time", time.time() - start)
        visits = [self.root.children[c].visits for c in self.root.children]
        actions = [a for a in self.root.children]
        #print("actions", actions)
        #print("visits", visits, len(visits))
        actions_p = softmax(1.0/temp * np.log(np.array(visits) + 1e-10))
        #print("visits", visits, np.array(visits).shape)
        #print("actions", actions_p)
        return actions, actions_p
    
    # perform one simulation fo mcts
    def traverse2(self, root, board, start):
        ptr = self.root
        while True:
            if len(ptr.children) == 0: break
            action, ptr = max(ptr.children.items(), key=lambda n: n[1].get_value())
            board.make_move(action)
        # base case, at leaf node
        #print("t before", time.time() - start)
        probs, value = self.pv_net.predict(board.get_state().reshape(-1, 4, 6, 6))
        #print("t after", time.time() - start)   
            
        if board.has_ended():
            #print("2")
            # use the true reward if game has ended
            if len(board.availables) == 0: value = 0
            else: value = board.player
        else:
            #print("3")
            #print("probs", probs.shape)
            ptr.expand(board.availables, probs[board.availables]) # expand the tree

        # backpropagate the value from leaf to root
        #print("value", value)
        ptr.update(-value)
    
         
    # perform one simulation fo mcts
    def traverse(self, root, board):
        # base case, at leaf node
        if len(root.children) == 0:
            #print("1")
            start = time.time()
            probs, value = self.pv_net.predict(board.get_state().reshape(-1, 4, 6, 6))
            #print(value)
            #print("t", time.time() - start)
            #print("p", probs)
            if board.has_ended():
                #print("2")
                #print(board.board)
                # use the true reward if game has ended
                if len(board.availables) == 0: value = 0
                else: value = -1
            else:
                #print("3")
                #print("probs", probs.shape)
                root.expand(board.availables, probs[board.availables]) # expand the tree
                
            # backpropagate the value from leaf to root
            #print("value", value)
            root.update(-value)
            return
            
        # Greedily select the children node with the max(Q + U) until leaf 
        #      where Q is the average of all values of nodes in the subtree and 
        #      U = c * p * sqrt(parent.visits) / (1 + visits)
        action, child = max(root.children.items(), key=lambda n: n[1].get_value())
        board.make_move(action)
        self.traverse(child, board)
    
        
    def forward(self, move):
        if move in self.root.children:
            self.root = self.root.children[move]
            self.root.parent = None
        else:
            self.root = Node(None, 1.0)
        
        
class Player():
    def __init__(self, pv_net, n_simulations=400):
        #print("p", pv_net)
        self.mcts = MCTS(pv_net, 1, n_simulations)
        self.n_simulations = n_simulations
        self.pv_net = pv_net
        
    def get_move(self, board, self_play=True):
        actions, actions_p = self.mcts.run(board)
        
        move = np.random.choice(
                    actions,
                    p=0.75*actions_p + 0.25*np.random.dirichlet(0.3*np.ones(len(actions_p)))
                )
                # update the root node and reuse the search tree
            
        if not self_play:
            move = actions[np.argmax(actions_p)]
            
        move_p = np.zeros(board.dimension**2)
        move_p[list(actions)] = actions_p
        if self_play:
            self.mcts.forward(move)
        else:
            self.mcts.forward(-1)
        return move, move_p
        
class Game():
    '''A Single Game of Gomoku'''
    def __init__(self, player, dimension, win_l, show):
        self.board = Board(dimension, win_l)
        self.states = []
        self.probs = []
        self.players = []
        self.opponent = {}
        self.player = player
        self.show = show
        
    # start game
    def start(self):
        i = 0
        #s = time.time()
        while True:
            move, p = self.player.get_move(self.board)
            i += 1
            #print("move", i, "time", time.time() - s)
            
            # before each move, save the the board state, probabilities of next actions and the current player at the turn
            #print("state after one move:\n", self.board.get_state())
            self.states.append(self.board.get_state())
            #print("current states", self.board.get_state())
            self.probs.append(p)
            #print("probs", p)
            self.players.append(self.board.player)
            #print("current player", self.board.player)
            
            self.board.make_move(move)
            if self.show:
                print(self.board.board)
            
            if self.board.has_ended():
                #print("1", self.board.board)
                winners = np.zeros(len(self.players))
                # someone wins
                if len(self.board.availables) > 0:
                    winner = -self.board.player
                    winners[np.array(self.players) == winner] = 1.0
                    winners[np.array(self.players) != winner] = -1.0
                '''
                for i in winners:
                    print("winner", i)
                for i in self.states:
                    print("s", i)
                for i in self.probs:
                    print("p", i)
                '''
                #print("winners", winners)
                return zip(self.states, self.probs, winners)
            
        
    def ended(self):
        return self.board.has_ended()
    
                
    

# run through the training pipeline
class Train():
    
    
    def __init__(self, dimension=6, win_l=4):
        self.dimension = dimension
        self.win_l = win_l
        self.pv_net = PVNet2(dimension, dimension)
        self.data_collection = deque(maxlen=10000)
        self.n_games = 300
        self.batch_size = 512
        self.epochs = 5
        self.lr_multiplier = 1.0
        self.kl_targ = 0.02
        self.data_len = []
        self.game_len = 0
        
    # collect data through self play
    def collect_data_self_play(self, show=False, n_rounds=1):
        # perform n rounds of self play
        for i in range(n_rounds):
            #print("collect", self.pv_net)
            player = Player(self.pv_net)
            game = Game(player, self.dimension, self.win_l, show)
            data = list(game.start())[:]
            #print(len(data))
            #self.data_len += len(data)
            #print(data)
            self.game_len = len(data)
            data = self.get_equi_data(data)
            self.data_len.append(len(data))
            #print("datalen", len(data))
            self.data_collection.extend(data)
      
    def get_equi_data(self, play_data):
        """augment the data set by rotation and flipping
        play_data: [(state, mcts_prob, winner_z), ..., ...]
        """
        extend_data = []
        for state, mcts_porb, winner in play_data:
            for i in [1, 2, 3, 4]:
                # rotate counterclockwise
                equi_state = np.array([np.rot90(s, i) for s in state])
                equi_mcts_prob = np.rot90(np.flipud(
                    mcts_porb.reshape(self.dimension, self.dimension)), i)
                extend_data.append((equi_state,
                                    np.flipud(equi_mcts_prob).flatten(),
                                    winner))
                # flip horizontally
                equi_state = np.array([np.fliplr(s) for s in equi_state])
                equi_mcts_prob = np.fliplr(equi_mcts_prob)
                extend_data.append((equi_state,
                                    np.flipud(equi_mcts_prob).flatten(),
                                    winner))
        return extend_data
    # run the training process
    def run(self, n_games= 300, display_games=50, l_rate_m=1.0):
        #self.lr_multiplier /= l_rate_m
        start = time.time()
        self.n_games = n_games
        for i in range(self.n_games):
            s = time.time() 
            self.collect_data_self_play()
            if ((i + 1) % display_games == 0):
                player = Player(self.pv_net, n_simulations=800)
                board = Board(self.dimension, self.win_l)
                while True:
                    move, p = player.get_move(board, self_play=False)
                    board.make_move(move)
                    pred = self.pv_net.predict(board.get_state().reshape(-1, 4, 6, 6))
                    print("prediction:\n", pred[0], "\n", pred[1])
                    print("p", p.reshape(-1, self.dimension))
                    print("move", move)
                    print("board\n", board.board)
                    if board.has_ended():
                        print(-board.player, "won")
                        break
 
            #print(time.time() - s)
            #if (len(self.data_collection)) > self.batch_size:
            print("game", i, "completed in", time.time() - s, "s", self.game_len, "steps")
            if (i >= 300):
                l = self.data_len.pop(0)
                #print("length", l, "data l", len(self.data_collection))
                for i in range(l):
                    self.data_collection.popleft()
                    
                print("data l after", len(self.data_collection))
            #print(self.data_collection[0])
            #print(states[0].shape)
            #print(probs[0].reshape(-1, self.dimension**2).shape)
            #print(winners[0].reshape(1, 1).shape, winners)
            #print(len(self.data_collection))
            if len(self.data_collection) > self.batch_size:
                s = time.time()
                # train neural network
                
                batch = random.sample(self.data_collection, self.batch_size)
                #s = np.array([d[0] for d in batch])
                #print("s1 shape", s.shape)
                states = np.array([d[0] for d in batch])
                #print("s shape", states.shape)
                probs = np.array([d[1] for d in batch])
                #print("p shape", probs.shape)
                winners = np.array([d[2] for d in batch]).reshape(-1, 1)
                #print("w shape", winners.shape)
                old_probs, old_v = self.pv_net.predict(states, v=False)
                #print(np.array(old_probs).shape, np.array(old_v).shape)
                for j in range(self.epochs):
                    #print(batch.shape)
                    
                    #print(states.shape, probs.shape, winners.shape)
                    loss, entropy = self.pv_net.train(states, probs, winners, 2e-3 * self.lr_multiplier)
                    
                    new_probs, new_v = self.pv_net.predict(states, v=False)
                    print("training", j, "lr_mult", self.lr_multiplier, "loss", loss, "entropy", entropy)
                    
                    kl = np.mean(np.sum(old_probs * (
                        np.log(old_probs + 1e-10) - np.log(new_probs + 1e-10)),
                        axis=1))
                    
                    if kl > self.kl_targ * 4:  # early stopping if D_KL diverges badly
                        break
                # adaptively adjust the learning rate
                print("kl", kl)
                
                if kl > self.kl_targ * 2 and self.lr_multiplier > 0.1:
                    self.lr_multiplier /= 1.5
                elif kl < self.kl_targ / 2 and self.lr_multiplier < 10:
                    self.lr_multiplier *= 1.5
                
                
                    
                print("completed in", time.time() - s, "s")
            
        print("training pipeline completed in", time.time() - start, "s")
    def save(self, path):
        self.pv_net.save(path)
            


In [5]:
%%time
from itertools import groupby
class Board():
    '''Board used in the Gomoku'''
    def __init__(self, dimension, win_l):
        self.dimension = dimension
        self.board = np.zeros((dimension, dimension))
        self.win_l = win_l         
        self.prev_move = -1
        self.player = 1
        self.availables = list(range(dimension * dimension))
        
    
    def make_move(self, pos):
        self.board[pos // self.dimension][pos % self.dimension] = self.player
        self.prev_move = pos
        self.player = -self.player
        self.availables.remove(pos)
        
    def has_won(self):
        def check(a):
            length = max(sum(1 for i in g) for k, g in groupby(a))
            #print("length", length)
            return length >= 4 and a[a.shape[0] // 2] != 0
            
            return False
        
        i = self.prev_move // self.dimension
        j = self.prev_move % self.dimension
        
        # horizontal
        #print("i", i, "j", j)
        #print("l", self.board[i, :])
        if check(self.board[i, :]): return True
        # vertical
        #print("2", self.board[:, j])
        if check(self.board[:, j]): return True
        # diagonal \
        #print("3", self.board.diagonal(j - i))
        if check(self.board.diagonal(j - i)): return True
        # diagonal /
        #print("4", np.fliplr(self.board).diagonal(self.dimension - j - i - 1))
        return check(np.fliplr(self.board).diagonal(self.dimension - j - i - 1))
    
    def has_ended(self):
        return len(self.availables) != self.dimension**2 and (self.has_won() or (len(self.availables) == 0))
    
    def get_state(self):
        #print(self.board.shape)
        state = np.zeros((4, self.board.shape[0], self.board.shape[1]))
        state[0] = (self.board == self.player).astype(int)
        state[1] = (self.board == -self.player).astype(int)
        last = np.zeros(self.board.shape)
        if self.prev_move != -1:
            last[self.prev_move // self.dimension][self.prev_move % self.dimension] = 1
        state[2] = last
        if self.player == 1:
            state[3] = np.ones(self.board.shape)
        else:
            state[3] = np.zeros(self.board.shape)
        
        return state
        

CPU times: user 52 µs, sys: 2 µs, total: 54 µs
Wall time: 58.9 µs


In [6]:
k = Train()

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_19454/3538894304.py:18: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/Users/fanrongqi/miniforge3/envs/tf/lib/python3.9/site-packages/keras/legacy_tf_layers/convolutional.py:563: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_19454/3538894304.py:22: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_19454/3538894304.py:26: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.k

Metal device set to: Apple M1 Max


In [7]:
k.run()

2022-10-12 18:44:09.389119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


game 0 completed in 28.175297021865845 s 29 steps
game 1 completed in 15.836066007614136 s 17 steps
game 2 completed in 19.57217812538147 s 20 steps


2022-10-12 18:45:13.761487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


training 0 lr_mult 1.0 loss 4.6010046 entropy 3.5826356
training 1 lr_mult 1.0 loss 4.590211 entropy 3.5829363
training 2 lr_mult 1.0 loss 4.5560665 entropy 3.580771
training 3 lr_mult 1.0 loss 4.497539 entropy 3.5761766
training 4 lr_mult 1.0 loss 4.4725895 entropy 3.568336
kl 0.0190255
completed in 4.596683025360107 s


KeyboardInterrupt: 

In [73]:
k.save('n2-300.model')

In [74]:
k.run()

game 0 completed in 6.848814964294434 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 3.2140422 entropy 2.545452
training 1 lr_mult 0.4444444444444444 loss 3.188348 entropy 2.5524023
training 2 lr_mult 0.4444444444444444 loss 3.1409156 entropy 2.5579104
training 3 lr_mult 0.4444444444444444 loss 3.0911727 entropy 2.5611782
training 4 lr_mult 0.4444444444444444 loss 3.0505993 entropy 2.5619135
kl 0.03351944
completed in 0.151777982711792 s
game 1 completed in 7.0047972202301025 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 3.1716425 entropy 2.536679
training 1 lr_mult 0.4444444444444444 loss 3.1491623 entropy 2.5413976
training 2 lr_mult 0.4444444444444444 loss 3.1175377 entropy 2.5499616
training 3 lr_mult 0.4444444444444444 loss 3.0847592 entropy 2.561005
training 4 lr_mult 0.4444444444444444 loss 3.0524225 entropy 2.5725088
kl 0.0228841
completed in 0.17209982872009277 s
game 2 completed in 11.82697319984436 s 14 steps
training 0 lr_mult 0.4444444444444444 loss 3.2179058 

game 19 completed in 13.88631796836853 s 17 steps
training 0 lr_mult 0.2962962962962963 loss 3.2230155 entropy 2.5521882
training 1 lr_mult 0.2962962962962963 loss 3.2061694 entropy 2.5523074
training 2 lr_mult 0.2962962962962963 loss 3.1794603 entropy 2.5498433
training 3 lr_mult 0.2962962962962963 loss 3.149455 entropy 2.545089
training 4 lr_mult 0.2962962962962963 loss 3.1208944 entropy 2.538546
kl 0.0075266855
completed in 0.1719820499420166 s
game 20 completed in 9.879530906677246 s 12 steps
training 0 lr_mult 0.4444444444444444 loss 3.160883 entropy 2.5153112
training 1 lr_mult 0.4444444444444444 loss 3.1274328 entropy 2.5052285
training 2 lr_mult 0.4444444444444444 loss 3.0808964 entropy 2.496113
training 3 lr_mult 0.4444444444444444 loss 3.0353274 entropy 2.489786
training 4 lr_mult 0.4444444444444444 loss 2.996563 entropy 2.4861484
kl 0.022934835
completed in 0.1714632511138916 s
game 21 completed in 7.308928966522217 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 3.0807

game 38 completed in 8.226725101470947 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 3.0386565 entropy 2.451143
training 1 lr_mult 0.4444444444444444 loss 3.0134397 entropy 2.459196
training 2 lr_mult 0.4444444444444444 loss 2.982628 entropy 2.4690008
training 3 lr_mult 0.4444444444444444 loss 2.9575484 entropy 2.4763017
training 4 lr_mult 0.4444444444444444 loss 2.9350567 entropy 2.4785008
kl 0.019155473
completed in 0.16772913932800293 s
game 39 completed in 7.974160671234131 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 3.040987 entropy 2.4938424
training 1 lr_mult 0.4444444444444444 loss 3.0128357 entropy 2.4851193
training 2 lr_mult 0.4444444444444444 loss 2.9712257 entropy 2.4724226
training 3 lr_mult 0.4444444444444444 loss 2.9347405 entropy 2.4578528
training 4 lr_mult 0.4444444444444444 loss 2.906094 entropy 2.4440284
kl 0.027115189
completed in 0.20631098747253418 s
game 40 completed in 8.095203876495361 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 3.

prediction:
 [0.0403441  0.01247142 0.00184208 0.00218243 0.00398805 0.00117222
 0.00654725 0.00970546 0.00591712 0.0071807  0.00151678 0.00359957
 0.00912337 0.0133544  0.00414293 0.21045648 0.01877537 0.00155742
 0.00204013 0.02541258 0.52184975 0.00188024 0.00916663 0.00366919
 0.00228239 0.00135598 0.00771423 0.00643575 0.0077242  0.00662646
 0.00083813 0.00464689 0.00230741 0.00276547 0.00816356 0.03124397] 
 0.26942465
p [[0.0350438  0.00375469 0.00125156 0.00250313 0.00250313 0.00125156]
 [0.00125156 0.         0.0738423  0.04630788 0.00375469 0.02252816]
 [0.00250313 0.08760951 0.         0.00625782 0.04505632 0.00250313]
 [0.00750939 0.0563204  0.01251564 0.         0.04881101 0.00125156]
 [0.00250313 0.01501877 0.02252816 0.08010013 0.36545682 0.00125156]
 [0.00250313 0.02002503 0.00250313 0.01627034 0.00250313 0.00500626]]
move 28
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.]
 [ 0

game 52 completed in 9.110831022262573 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 3.0357974 entropy 2.449979
training 1 lr_mult 0.4444444444444444 loss 3.0133162 entropy 2.4405642
training 2 lr_mult 0.4444444444444444 loss 2.973337 entropy 2.427888
training 3 lr_mult 0.4444444444444444 loss 2.9264007 entropy 2.4127903
training 4 lr_mult 0.4444444444444444 loss 2.8779933 entropy 2.395927
kl 0.024657821
completed in 0.2037949562072754 s
game 53 completed in 9.152209043502808 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 3.1022792 entropy 2.3927412
training 1 lr_mult 0.4444444444444444 loss 3.0789633 entropy 2.3824441
training 2 lr_mult 0.4444444444444444 loss 3.0335555 entropy 2.3755898
training 3 lr_mult 0.4444444444444444 loss 2.986228 entropy 2.3713956
training 4 lr_mult 0.4444444444444444 loss 2.9469419 entropy 2.3688996
kl 0.031923838
completed in 0.19950008392333984 s
game 54 completed in 9.933630228042603 s 12 steps
training 0 lr_mult 0.4444444444444444 loss 3.0

...

prediction:
 [5.08788833e-03 3.99620412e-03 2.23839434e-05 2.13010376e-03
 5.86315384e-03 9.59804820e-05 1.96743861e-01 3.78849596e-04
 3.47968340e-02 9.05145146e-03 4.10207324e-02 7.29242293e-03
 1.53676607e-03 1.61784366e-04 8.62597790e-06 1.19164184e-01
 3.49118462e-04 1.60447002e-04 1.01009653e-04 8.36777544e-05
 3.52091459e-03 3.45687454e-06 1.10850888e-04 8.31249286e-04
 9.93443094e-03 1.16052590e-02 5.21849084e-04 3.44810169e-03
 3.73771938e-04 5.04638731e-01 1.97322879e-04 2.34410372e-02
 6.56809891e-03 6.06233334e-05 5.25184395e-03 1.44713733e-03] 
 0.8479192
p [[0.0212766  0.00500626 0.00250313 0.00500626 0.00500626 0.00625782]
 [0.02002503 0.         0.01627034 0.00500626 0.03754693 0.01001252]
 [0.00876095 0.06257822 0.         0.         0.01251564 0.00250313]
 [0.00500626 0.         0.0563204  0.         0.25782228 0.00375469]
 [0.01877347 0.12765957 0.         0.         0.         0.25031289]
 [0.00375469 0.05006258 0.00625782 0.         0.         0.        ]]
move 22


game 262 completed in 9.32127594947815 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.5856566 entropy 2.1084619
training 1 lr_mult 0.4444444444444444 loss 2.5669909 entropy 2.0955787
training 2 lr_mult 0.4444444444444444 loss 2.5377288 entropy 2.0841186
training 3 lr_mult 0.4444444444444444 loss 2.5082974 entropy 2.07591
training 4 lr_mult 0.4444444444444444 loss 2.477091 entropy 2.0709355
kl 0.0265444
completed in 0.16678404808044434 s
game 263 completed in 13.857203960418701 s 17 steps
training 0 lr_mult 0.4444444444444444 loss 2.6282432 entropy 2.0875502
training 1 lr_mult 0.4444444444444444 loss 2.6045766 entropy 2.0907617
training 2 lr_mult 0.4444444444444444 loss 2.5723045 entropy 2.0970197
training 3 lr_mult 0.4444444444444444 loss 2.5362947 entropy 2.1022553
training 4 lr_mult 0.4444444444444444 loss 2.498373 entropy 2.1046157
kl 0.017068835
completed in 0.16910696029663086 s
game 264 completed in 6.022594690322876 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2

game 281 completed in 11.031743049621582 s 13 steps
training 0 lr_mult 0.2962962962962963 loss 2.6013415 entropy 2.1233885
training 1 lr_mult 0.2962962962962963 loss 2.5847204 entropy 2.1251657
training 2 lr_mult 0.2962962962962963 loss 2.55741 entropy 2.1267004
training 3 lr_mult 0.2962962962962963 loss 2.5261445 entropy 2.1269054
training 4 lr_mult 0.2962962962962963 loss 2.4936347 entropy 2.1248481
kl 0.015336077
completed in 0.20610427856445312 s
game 282 completed in 8.3087899684906 s 10 steps
training 0 lr_mult 0.2962962962962963 loss 2.631807 entropy 2.0763586
training 1 lr_mult 0.2962962962962963 loss 2.6116664 entropy 2.072174
training 2 lr_mult 0.2962962962962963 loss 2.5789921 entropy 2.0685287
training 3 lr_mult 0.2962962962962963 loss 2.5413744 entropy 2.0658119
training 4 lr_mult 0.2962962962962963 loss 2.5056136 entropy 2.0641174
kl 0.026286768
completed in 0.1688098907470703 s
game 283 completed in 5.951807975769043 s 7 steps
training 0 lr_mult 0.2962962962962963 loss 2

prediction:
 [4.9202732e-04 5.2733999e-04 1.6968186e-03 7.0656795e-04 1.1985108e-03
 1.8096420e-04 4.7400911e-04 6.7983097e-03 3.6661613e-03 3.7877053e-02
 1.9623837e-04 7.0427795e-04 1.8165377e-03 2.1478762e-03 1.0774465e-02
 3.5906377e-01 4.6420481e-02 2.7092596e-04 4.7018312e-04 8.5663453e-02
 3.4369740e-01 8.3138691e-03 3.2637394e-03 1.3117720e-03 5.0965056e-04
 2.3012228e-04 6.5718897e-02 4.0166206e-03 5.2905721e-03 4.9803592e-04
 4.4111945e-04 1.0997974e-03 1.3481042e-03 1.4853830e-03 4.8096839e-04
 1.1480899e-03] 
 0.51708883
p [[0.00250313 0.01501877 0.00375469 0.00876095 0.00375469 0.00250313]
 [0.12891114 0.07634543 0.04630788 0.03754693 0.00500626 0.00500626]
 [0.00375469 0.04380476 0.23404255 0.02252816 0.03379224 0.00625782]
 [0.00625782 0.04380476 0.01126408 0.         0.05131414 0.00625782]
 [0.00375469 0.00500626 0.02628285 0.08760951 0.02503129 0.01501877]
 [0.00500626 0.00375469 0.00625782 0.00250313 0.01627034 0.00500626]]
move 14
board
 [[ 0.  0.  0.  0.  0.  0.]
 [

prediction:
 [4.6840278e-03 6.4613141e-02 2.4673678e-02 4.6678139e-03 9.8886723e-03
 1.6038894e-03 3.4164585e-02 1.6295316e-02 1.6286826e-01 1.4500647e-03
 8.5270760e-04 7.6293293e-03 4.8943092e-03 3.3613160e-04 2.0679575e-04
 3.9806582e-05 4.1325178e-02 1.2872241e-02 2.7558448e-02 5.1297635e-02
 4.1584270e-05 4.7892925e-05 7.9474418e-04 5.2581285e-03 1.0574269e-02
 1.5348181e-03 1.5248243e-03 2.6604185e-01 1.1944242e-02 8.4615499e-02
 5.3542270e-03 6.5473113e-03 7.4419640e-03 8.3574958e-02 3.2712925e-02
 1.0068717e-02] 
 -0.18350689
p [[1.25156446e-03 2.12765957e-02 7.50938673e-03 2.25281602e-02
  2.12765957e-02 1.50187735e-02]
 [1.12640801e-02 1.62703379e-02 1.37672090e-02 0.00000000e+00
  3.75469337e-03 3.87984981e-02]
 [1.37672090e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  5.00625782e-03 6.42052566e-01]
 [2.25281602e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.25281602e-02 1.25156446e-13 5.00625782e-03 6.25782228e-03
  7.50938673e-03

In [75]:
k.save('n2-600.model')

In [76]:
k.run()

game 0 completed in 12.800676107406616 s 15 steps
training 0 lr_mult 0.2962962962962963 loss 2.6192636 entropy 2.073866
training 1 lr_mult 0.2962962962962963 loss 2.6081326 entropy 2.0686808
training 2 lr_mult 0.2962962962962963 loss 2.584226 entropy 2.0655746
training 3 lr_mult 0.2962962962962963 loss 2.5533473 entropy 2.0644102
training 4 lr_mult 0.2962962962962963 loss 2.52252 entropy 2.0648923
kl 0.01311289
completed in 0.1706070899963379 s
game 1 completed in 8.838735103607178 s 10 steps
training 0 lr_mult 0.2962962962962963 loss 2.764078 entropy 2.1017222
training 1 lr_mult 0.2962962962962963 loss 2.7409296 entropy 2.1060333
training 2 lr_mult 0.2962962962962963 loss 2.7062967 entropy 2.11161
training 3 lr_mult 0.2962962962962963 loss 2.6672606 entropy 2.1171317
training 4 lr_mult 0.2962962962962963 loss 2.630884 entropy 2.1217299
kl 0.021925766
completed in 0.17171502113342285 s
game 2 completed in 6.216421127319336 s 7 steps
training 0 lr_mult 0.2962962962962963 loss 2.640523 e

game 19 completed in 7.328654050827026 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.644983 entropy 2.0547194
training 1 lr_mult 0.4444444444444444 loss 2.6225383 entropy 2.0510793
training 2 lr_mult 0.4444444444444444 loss 2.5867107 entropy 2.0500941
training 3 lr_mult 0.4444444444444444 loss 2.5459042 entropy 2.0513134
training 4 lr_mult 0.4444444444444444 loss 2.5083132 entropy 2.0539665
kl 0.027985599
completed in 0.1754591464996338 s
game 20 completed in 6.297815799713135 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.5295963 entropy 2.0494068
training 1 lr_mult 0.4444444444444444 loss 2.513777 entropy 2.0502753
training 2 lr_mult 0.4444444444444444 loss 2.4842877 entropy 2.0504818
training 3 lr_mult 0.4444444444444444 loss 2.4467196 entropy 2.0501657
training 4 lr_mult 0.4444444444444444 loss 2.4078069 entropy 2.048974
kl 0.023629313
completed in 0.22082996368408203 s
game 21 completed in 10.829740047454834 s 13 steps
training 0 lr_mult 0.4444444444444444 loss 2.

training 4 lr_mult 0.2962962962962963 loss 2.5006797 entropy 2.0127861
kl 0.01501266
completed in 0.20415067672729492 s
game 38 completed in 6.113832235336304 s 7 steps
training 0 lr_mult 0.2962962962962963 loss 2.6669228 entropy 2.0809808
training 1 lr_mult 0.2962962962962963 loss 2.644621 entropy 2.089387
training 2 lr_mult 0.2962962962962963 loss 2.6141193 entropy 2.0985403
training 3 lr_mult 0.2962962962962963 loss 2.5866559 entropy 2.1066704
training 4 lr_mult 0.2962962962962963 loss 2.5596805 entropy 2.1116915
kl 0.016761087
completed in 0.17518997192382812 s
game 39 completed in 12.777076959609985 s 15 steps
training 0 lr_mult 0.2962962962962963 loss 2.6759417 entropy 2.087523
training 1 lr_mult 0.2962962962962963 loss 2.6471612 entropy 2.0870013
training 2 lr_mult 0.2962962962962963 loss 2.6178582 entropy 2.086155
training 3 lr_mult 0.2962962962962963 loss 2.5915282 entropy 2.086586
training 4 lr_mult 0.2962962962962963 loss 2.5646093 entropy 2.0882015
kl 0.01672588
completed i

prediction:
 [8.6330558e-04 1.0873686e-03 2.3519606e-04 7.1719737e-04 1.5213505e-04
 7.7657543e-02 1.3107772e-03 3.5040017e-04 5.0291931e-03 3.1605293e-04
 2.7503937e-01 2.2080576e-04 3.0191767e-04 3.6737702e-03 7.4179785e-05
 2.5512450e-04 3.0650609e-04 1.8309403e-04 1.6161428e-04 4.9254001e-04
 5.3778058e-04 4.9847138e-05 3.8846529e-03 4.4316723e-04 3.3486713e-04
 5.4119021e-01 5.0413754e-04 7.7658277e-03 4.8919127e-04 1.0457825e-03
 7.2732016e-02 1.7406543e-04 3.3281840e-04 2.2506565e-04 1.2434452e-03
 6.1914814e-04] 
 -0.9745876
p [[0.00125156 0.00125156 0.00125156 0.00125156 0.00250313 0.00125156]
 [0.00125156 0.00250313 0.00625782 0.02753442 0.15394243 0.00250313]
 [0.00250313 0.00250313 0.41301627 0.         0.04505632 0.00125156]
 [0.00125156 0.01501877 0.         0.         0.00500626 0.00125156]
 [0.00500626 0.25031289 0.03254068 0.00750939 0.00125156 0.00125156]
 [0.00250313 0.00375469 0.00125156 0.00250313 0.00125156 0.00125156]]
move 14
board
 [[ 0.  0.  0.  0.  0.  0.]
 [

training 4 lr_mult 0.2962962962962963 loss 2.4795163 entropy 2.0865464
kl 0.012830953
completed in 0.20483708381652832 s
game 57 completed in 8.495727062225342 s 10 steps
training 0 lr_mult 0.2962962962962963 loss 2.6137822 entropy 2.0794072
training 1 lr_mult 0.2962962962962963 loss 2.5925682 entropy 2.0776474
training 2 lr_mult 0.2962962962962963 loss 2.564396 entropy 2.077855
training 3 lr_mult 0.2962962962962963 loss 2.5355768 entropy 2.0790732
training 4 lr_mult 0.2962962962962963 loss 2.5086987 entropy 2.0805254
kl 0.01488098
completed in 0.2031850814819336 s
game 58 completed in 13.24885082244873 s 16 steps
training 0 lr_mult 0.2962962962962963 loss 2.589507 entropy 2.0785818
training 1 lr_mult 0.2962962962962963 loss 2.570618 entropy 2.0775561
training 2 lr_mult 0.2962962962962963 loss 2.544101 entropy 2.0751677
training 3 lr_mult 0.2962962962962963 loss 2.516523 entropy 2.0725222
training 4 lr_mult 0.2962962962962963 loss 2.4875016 entropy 2.069789
kl 0.0120421965
completed in

...

prediction:
 [4.16748109e-04 2.36256093e-01 1.17091532e-03 2.41260529e-02
 9.50842339e-04 1.95990643e-03 1.43972561e-01 8.47141549e-04
 1.67341219e-04 7.03847548e-03 1.50751742e-02 1.94789330e-03
 6.12424046e-04 6.30924255e-02 1.07912021e-02 3.74825322e-04
 4.04543895e-03 3.80038889e-03 1.89766183e-03 3.25511512e-03
 2.95317208e-04 1.99957117e-02 5.20622395e-02 6.12389005e-04
 3.81700648e-03 1.44776264e-02 9.41247772e-03 2.64147413e-04
 5.06754732e-04 1.20785415e-01 4.63498803e-03 1.04847422e-03
 1.37432218e-02 1.05329684e-03 2.35269442e-01 2.23069816e-04] 
 -0.87095934
p [[1.25156446e-13 1.00125156e-02 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.00125156e-02 1.25156446e-13 9.26157697e-01 1.25156446e-03
  1.25156446e-03 1.25156446e-13]
 [1.25156446e-13 5.00625782e-03 1.25156446e-03 0.00000000e+00
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 0.00000000e+00 1.25156446e-03
  0.00000000e+00 1.25156446e-13]
 [1.25156446e-13 2.12765957e-02 1.2515644

game 259 completed in 7.553842782974243 s 9 steps
training 0 lr_mult 0.2962962962962963 loss 2.4484456 entropy 1.8591423
training 1 lr_mult 0.2962962962962963 loss 2.4379423 entropy 1.8558104
training 2 lr_mult 0.2962962962962963 loss 2.4216151 entropy 1.8537865
training 3 lr_mult 0.2962962962962963 loss 2.4009483 entropy 1.8526502
training 4 lr_mult 0.2962962962962963 loss 2.3773348 entropy 1.851801
kl 0.014970458
completed in 0.17533111572265625 s
game 260 completed in 14.013243913650513 s 17 steps
training 0 lr_mult 0.2962962962962963 loss 2.4130576 entropy 1.8318286
training 1 lr_mult 0.2962962962962963 loss 2.4048588 entropy 1.8305337
training 2 lr_mult 0.2962962962962963 loss 2.3890696 entropy 1.8297215
training 3 lr_mult 0.2962962962962963 loss 2.3679774 entropy 1.829123
training 4 lr_mult 0.2962962962962963 loss 2.3455453 entropy 1.8283862
kl 0.01264745
completed in 0.17491507530212402 s
game 261 completed in 9.165637016296387 s 11 steps
training 0 lr_mult 0.2962962962962963 lo

game 278 completed in 11.522608280181885 s 14 steps
training 0 lr_mult 0.2962962962962963 loss 2.4861364 entropy 1.8723001
training 1 lr_mult 0.2962962962962963 loss 2.4726708 entropy 1.8696587
training 2 lr_mult 0.2962962962962963 loss 2.449905 entropy 1.8671069
training 3 lr_mult 0.2962962962962963 loss 2.4236958 entropy 1.8645048
training 4 lr_mult 0.2962962962962963 loss 2.397246 entropy 1.8616632
kl 0.020304183
completed in 0.1767590045928955 s
game 279 completed in 7.421741962432861 s 9 steps
training 0 lr_mult 0.2962962962962963 loss 2.3679345 entropy 1.8232608
training 1 lr_mult 0.2962962962962963 loss 2.357092 entropy 1.8212087
training 2 lr_mult 0.2962962962962963 loss 2.3376746 entropy 1.8212744
training 3 lr_mult 0.2962962962962963 loss 2.3114414 entropy 1.8234628
training 4 lr_mult 0.2962962962962963 loss 2.2833836 entropy 1.82704
kl 0.01575468
completed in 0.1732630729675293 s
game 280 completed in 7.324859857559204 s 9 steps
training 0 lr_mult 0.2962962962962963 loss 2.3

game 297 completed in 8.315414905548096 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 2.547793 entropy 1.8993032
training 1 lr_mult 0.4444444444444444 loss 2.5234718 entropy 1.902842
training 2 lr_mult 0.4444444444444444 loss 2.4843943 entropy 1.9080468
training 3 lr_mult 0.4444444444444444 loss 2.4477575 entropy 1.9127032
training 4 lr_mult 0.4444444444444444 loss 2.4148774 entropy 1.9142312
kl 0.03033007
completed in 0.1517329216003418 s
game 298 completed in 6.024514675140381 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.548251 entropy 1.875042
training 1 lr_mult 0.4444444444444444 loss 2.508288 entropy 1.8650866
training 2 lr_mult 0.4444444444444444 loss 2.4690595 entropy 1.8523148
training 3 lr_mult 0.4444444444444444 loss 2.4358635 entropy 1.8401423
training 4 lr_mult 0.4444444444444444 loss 2.4015388 entropy 1.8306931
kl 0.035730496
completed in 0.17676711082458496 s
prediction:
 [0.0015619  0.00303395 0.00192983 0.00096292 0.00366698 0.00147191
 0.00282058 0.00

prediction:
 [0.00376171 0.0031222  0.01884435 0.00525628 0.00133204 0.00210119
 0.00160429 0.0262796  0.07854544 0.19585618 0.00293264 0.00247549
 0.00281622 0.02431492 0.00498453 0.00184975 0.0456234  0.01143369
 0.0087272  0.03632573 0.00293147 0.00321283 0.01128192 0.00625665
 0.00323898 0.0071169  0.3460487  0.05876701 0.02327903 0.00156174
 0.00195236 0.00351024 0.01022099 0.03792804 0.0016058  0.00290056] 
 0.86176074
p [[0.00125156 0.05506884 0.00250313 0.         0.00125156 0.00125156]
 [0.00500626 0.00250313 0.00375469 0.         0.00750939 0.00125156]
 [0.00125156 0.         0.78598248 0.         0.00500626 0.0175219 ]
 [0.00876095 0.00250313 0.         0.         0.00750939 0.00125156]
 [0.00125156 0.00625782 0.00876095 0.         0.00375469 0.00625782]
 [0.00125156 0.00125156 0.00250313 0.00750939 0.04881101 0.00125156]]
move 14
board
 [[ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  1. -1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0

In [77]:
k.save('n2-900.model')

In [78]:
k.run()

game 0 completed in 7.701918363571167 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.4605017 entropy 1.7951505
training 1 lr_mult 0.4444444444444444 loss 2.4431546 entropy 1.7925993
training 2 lr_mult 0.4444444444444444 loss 2.420861 entropy 1.7913243
training 3 lr_mult 0.4444444444444444 loss 2.3939025 entropy 1.7927802
training 4 lr_mult 0.4444444444444444 loss 2.3565302 entropy 1.797054
kl 0.01848074
completed in 0.17058610916137695 s
game 1 completed in 7.923114061355591 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.5128202 entropy 1.8340323
training 1 lr_mult 0.4444444444444444 loss 2.4929202 entropy 1.8421507
training 2 lr_mult 0.4444444444444444 loss 2.4605138 entropy 1.8501991
training 3 lr_mult 0.4444444444444444 loss 2.4220643 entropy 1.8573993
training 4 lr_mult 0.4444444444444444 loss 2.382882 entropy 1.8633251
kl 0.026909605
completed in 0.1872708797454834 s
game 2 completed in 7.686304807662964 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.379864

game 19 completed in 13.625030040740967 s 17 steps
training 0 lr_mult 0.2962962962962963 loss 2.4233267 entropy 1.8278303
training 1 lr_mult 0.2962962962962963 loss 2.4086506 entropy 1.8279686
training 2 lr_mult 0.2962962962962963 loss 2.3897223 entropy 1.826648
training 3 lr_mult 0.2962962962962963 loss 2.3715432 entropy 1.8248178
training 4 lr_mult 0.2962962962962963 loss 2.3517694 entropy 1.8231778
kl 0.010999002
completed in 0.17678093910217285 s
game 20 completed in 12.126585006713867 s 15 steps
training 0 lr_mult 0.2962962962962963 loss 2.413467 entropy 1.8140309
training 1 lr_mult 0.2962962962962963 loss 2.39737 entropy 1.814783
training 2 lr_mult 0.2962962962962963 loss 2.3751683 entropy 1.8165462
training 3 lr_mult 0.2962962962962963 loss 2.3563766 entropy 1.8183714
training 4 lr_mult 0.2962962962962963 loss 2.3379447 entropy 1.8194184
kl 0.008822982
completed in 0.16971683502197266 s
game 21 completed in 9.933645009994507 s 12 steps
training 0 lr_mult 0.4444444444444444 loss 

game 38 completed in 8.694772958755493 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.5337675 entropy 1.8675046
training 1 lr_mult 0.4444444444444444 loss 2.515778 entropy 1.8659246
training 2 lr_mult 0.4444444444444444 loss 2.4849231 entropy 1.8619754
training 3 lr_mult 0.4444444444444444 loss 2.4476476 entropy 1.856627
training 4 lr_mult 0.4444444444444444 loss 2.409038 entropy 1.8508387
kl 0.031949487
completed in 0.1884140968322754 s
game 39 completed in 7.775240898132324 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.5009592 entropy 1.798028
training 1 lr_mult 0.4444444444444444 loss 2.4865723 entropy 1.7970889
training 2 lr_mult 0.4444444444444444 loss 2.4621897 entropy 1.8001863
training 3 lr_mult 0.4444444444444444 loss 2.4342747 entropy 1.8056802
training 4 lr_mult 0.4444444444444444 loss 2.4057322 entropy 1.8117385
kl 0.03216619
completed in 0.16372919082641602 s
game 40 completed in 6.354335784912109 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.4937

prediction:
 [9.90694156e-04 1.83853772e-04 5.39206667e-04 1.41540857e-03
 9.23795305e-05 8.19403795e-04 6.01707550e-04 2.68912339e-03
 2.17719853e-01 1.10398298e-02 3.38875386e-03 1.75051435e-04
 1.10835826e-03 1.06492182e-02 1.76037848e-02 8.44308175e-04
 2.34664366e-01 1.00944899e-02 1.03248255e-02 2.43452951e-01
 5.98985120e-04 1.90795865e-02 1.14367716e-02 9.88710206e-04
 1.40985605e-04 1.17694749e-03 1.31106600e-02 1.77028447e-01
 3.21494066e-03 7.08193402e-04 5.90688782e-04 1.51356187e-04
 2.41584936e-03 5.73077297e-04 1.26779327e-04 2.60585715e-04] 
 0.7794943
p [[0.00125156 0.00125156 0.00125156 0.00125156 0.00125156 0.00125156]
 [0.00250313 0.00250313 0.00250313 0.00125156 0.00125156 0.00125156]
 [0.00125156 0.7008761  0.         0.         0.00876095 0.00125156]
 [0.00125156 0.00876095 0.         0.00250313 0.23153942 0.00125156]
 [0.00125156 0.00250313 0.00250313 0.00625782 0.00125156 0.00125156]
 [0.00125156 0.00125156 0.00250313 0.00125156 0.00125156 0.00125156]]
move 13


prediction:
 [0.0016976  0.02556029 0.049605   0.02061562 0.01150292 0.00114167
 0.02129089 0.02901284 0.04904469 0.06275816 0.0067971  0.16155466
 0.02793477 0.00525004 0.00509142 0.00063975 0.00750336 0.01020333
 0.00563619 0.0089444  0.00058607 0.00300326 0.00306572 0.01655204
 0.09978965 0.01344152 0.19597146 0.05271645 0.02242895 0.01386849
 0.00331929 0.02847089 0.00816632 0.01587864 0.00912145 0.00183508] 
 -0.85191244
p [[0.01501877 0.0350438  0.00500626 0.05381727 0.00250313 0.00125156]
 [0.13266583 0.10763454 0.04755945 0.27534418 0.00125156 0.00125156]
 [0.00125156 0.         0.         0.         0.         0.        ]
 [0.02002503 0.         0.         0.         0.         0.        ]
 [0.00250313 0.04255319 0.03003755 0.         0.02252816 0.16520651]
 [0.00125156 0.00250313 0.00375469 0.00125156 0.02252816 0.00625782]]
move 9
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0. -1.  1.  1.  1. -1.]
 [ 0.  1. -1. -1. -1.  1.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0

...

prediction:
 [2.73845188e-04 3.52678180e-05 6.21128478e-04 1.49618805e-04
 1.05718485e-04 4.01471480e-05 1.07156506e-04 3.40158050e-03
 3.02966125e-03 9.49619487e-02 1.86844351e-04 1.03425344e-04
 3.88900255e-04 1.42976211e-03 4.88851815e-02 4.81156707e-01
 3.98362838e-02 3.07945273e-04 2.78056308e-04 2.76915245e-02
 1.29781991e-01 5.51271066e-02 2.68566213e-03 1.62780590e-04
 1.57584989e-04 1.99596310e-04 1.01360328e-01 3.13902786e-03
 2.67032580e-03 4.08936394e-05 5.17285771e-05 7.51589105e-05
 3.73597839e-04 7.94207735e-04 5.50078876e-05 3.34359764e-04] 
 0.5853337
p [[0.00125156 0.00125156 0.00250313 0.00125156 0.00125156 0.00125156]
 [0.00250313 0.2077597  0.12765957 0.01251564 0.00750939 0.00250313]
 [0.00500626 0.10513141 0.12015019 0.01001252 0.00500626 0.00250313]
 [0.00250313 0.0212766  0.00750939 0.         0.08760951 0.00125156]
 [0.00125156 0.00750939 0.00750939 0.02628285 0.19649562 0.00125156]
 [0.00125156 0.00625782 0.00876095 0.00375469 0.00125156 0.00125156]]
move 7
b

prediction:
 [1.9520207e-04 7.0850918e-05 1.0280042e-03 1.0163479e-03 5.7858310e-04
 3.4946252e-02 1.3218491e-04 4.9804233e-04 6.3057328e-03 8.2030252e-04
 5.4472539e-04 1.0054881e-03 7.0671755e-04 2.5059384e-01 3.6114245e-04
 1.7286920e-04 1.8853979e-04 2.4861467e-04 2.1300639e-04 8.6010760e-04
 1.6312356e-04 1.0016931e-04 6.3106406e-01 4.9682646e-03 1.5235844e-03
 1.1170993e-02 8.4696067e-03 3.7806677e-03 3.4417887e-04 4.6011401e-04
 3.4874354e-02 8.4264507e-04 1.3159295e-03 1.4141841e-04 2.5636606e-05
 2.6884297e-04] 
 -0.7634116
p [[0.00125156 0.12265332 0.0387985  0.         0.00125156 0.01251564]
 [0.00125156 0.         0.03254068 0.         0.         0.00375469]
 [0.00125156 0.21401752 0.00125156 0.         0.00125156 0.00125156]
 [0.00125156 0.         0.         0.         0.23404255 0.00250313]
 [0.00125156 0.27534418 0.00125156 0.         0.00250313 0.00250313]
 [0.03379224 0.00125156 0.00125156 0.00250313 0.00250313 0.00500626]]
move 25
board
 [[ 0.  0.  0. -1.  0.  0.]
 [

game 262 completed in 8.410836935043335 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 2.3854876 entropy 1.8571277
training 1 lr_mult 0.4444444444444444 loss 2.3659015 entropy 1.8549764
training 2 lr_mult 0.4444444444444444 loss 2.3360636 entropy 1.8568182
training 3 lr_mult 0.4444444444444444 loss 2.3042052 entropy 1.8614719
training 4 lr_mult 0.4444444444444444 loss 2.2721586 entropy 1.8669744
kl 0.022869423
completed in 0.17188501358032227 s
game 263 completed in 7.528733968734741 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.530523 entropy 1.9345531
training 1 lr_mult 0.4444444444444444 loss 2.5139642 entropy 1.9402881
training 2 lr_mult 0.4444444444444444 loss 2.4869416 entropy 1.9427052
training 3 lr_mult 0.4444444444444444 loss 2.4587796 entropy 1.9413307
training 4 lr_mult 0.4444444444444444 loss 2.432857 entropy 1.9368913
kl 0.016020138
completed in 0.1665809154510498 s
game 264 completed in 13.734796047210693 s 17 steps
training 0 lr_mult 0.4444444444444444 lo

training 4 lr_mult 0.4444444444444444 loss 2.2127917 entropy 1.81527
kl 0.020596767
completed in 0.20254015922546387 s
game 281 completed in 7.027614116668701 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.4780254 entropy 1.8401852
training 1 lr_mult 0.4444444444444444 loss 2.462246 entropy 1.8387185
training 2 lr_mult 0.4444444444444444 loss 2.4268107 entropy 1.8403566
training 3 lr_mult 0.4444444444444444 loss 2.383751 entropy 1.8436197
training 4 lr_mult 0.4444444444444444 loss 2.3425179 entropy 1.8472314
kl 0.019939864
completed in 0.19789576530456543 s
game 282 completed in 7.992854833602905 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.3594804 entropy 1.8567017
training 1 lr_mult 0.4444444444444444 loss 2.346269 entropy 1.8604043
training 2 lr_mult 0.4444444444444444 loss 2.3261113 entropy 1.8644636
training 3 lr_mult 0.4444444444444444 loss 2.3035297 entropy 1.8686318
training 4 lr_mult 0.4444444444444444 loss 2.2824416 entropy 1.8727435
kl 0.021741647
completed

prediction:
 [0.00542614 0.0038856  0.00286673 0.00663045 0.00214378 0.00365142
 0.00480343 0.21545747 0.0473316  0.01526191 0.011341   0.00217772
 0.00353036 0.0464882  0.07296404 0.01512908 0.02093402 0.01135905
 0.00639017 0.01438961 0.01911207 0.09223121 0.04731594 0.00459612
 0.00220747 0.00897793 0.01424284 0.05037989 0.21865736 0.00324645
 0.00199145 0.00289575 0.00818062 0.00275387 0.00458274 0.00646664] 
 -0.71591157
p [[1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 2.65331665e-01 2.36545682e-01
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 2.39048811e-01 2.59073842e-01
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13

In [80]:
k.save('n2-1200.model')

In [81]:
k.run()

game 0 completed in 7.50545597076416 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.4060495 entropy 1.8667347
training 1 lr_mult 0.4444444444444444 loss 2.392938 entropy 1.8682678
training 2 lr_mult 0.4444444444444444 loss 2.3661366 entropy 1.870348
training 3 lr_mult 0.4444444444444444 loss 2.335247 entropy 1.8715175
training 4 lr_mult 0.4444444444444444 loss 2.3052037 entropy 1.871023
kl 0.019948922
completed in 0.17284083366394043 s
game 1 completed in 7.47498083114624 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.327504 entropy 1.8818567
training 1 lr_mult 0.4444444444444444 loss 2.310401 entropy 1.8786076
training 2 lr_mult 0.4444444444444444 loss 2.2829447 entropy 1.8741794
training 3 lr_mult 0.4444444444444444 loss 2.257152 entropy 1.8705096
training 4 lr_mult 0.4444444444444444 loss 2.2343912 entropy 1.8687543
kl 0.026166867
completed in 0.17522263526916504 s
game 2 completed in 7.580127954483032 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.322889 ent

game 19 completed in 10.580837965011597 s 13 steps
training 0 lr_mult 0.4444444444444444 loss 2.317258 entropy 1.8165792
training 1 lr_mult 0.4444444444444444 loss 2.3017101 entropy 1.8154633
training 2 lr_mult 0.4444444444444444 loss 2.2766387 entropy 1.8139884
training 3 lr_mult 0.4444444444444444 loss 2.250312 entropy 1.8121455
training 4 lr_mult 0.4444444444444444 loss 2.226294 entropy 1.8100573
kl 0.016232762
completed in 0.17157721519470215 s
game 20 completed in 5.884858131408691 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.3298535 entropy 1.8155799
training 1 lr_mult 0.4444444444444444 loss 2.3087778 entropy 1.8146746
training 2 lr_mult 0.4444444444444444 loss 2.282509 entropy 1.8151174
training 3 lr_mult 0.4444444444444444 loss 2.255789 entropy 1.8173623
training 4 lr_mult 0.4444444444444444 loss 2.2293239 entropy 1.8205516
kl 0.021800946
completed in 0.20227718353271484 s
game 21 completed in 12.19047212600708 s 15 steps
training 0 lr_mult 0.4444444444444444 loss 2.

game 38 completed in 9.268835067749023 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.4234233 entropy 1.8598492
training 1 lr_mult 0.4444444444444444 loss 2.390618 entropy 1.8513565
training 2 lr_mult 0.4444444444444444 loss 2.351682 entropy 1.841669
training 3 lr_mult 0.4444444444444444 loss 2.3218615 entropy 1.8352723
training 4 lr_mult 0.4444444444444444 loss 2.2917285 entropy 1.8320441
kl 0.018633341
completed in 0.1748948097229004 s
game 39 completed in 7.572691202163696 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.431999 entropy 1.8271692
training 1 lr_mult 0.4444444444444444 loss 2.4134781 entropy 1.8306473
training 2 lr_mult 0.4444444444444444 loss 2.386154 entropy 1.8365798
training 3 lr_mult 0.4444444444444444 loss 2.3532603 entropy 1.843919
training 4 lr_mult 0.4444444444444444 loss 2.3192194 entropy 1.8526696
kl 0.024366919
completed in 0.18933987617492676 s
game 40 completed in 15.522583961486816 s 19 steps
training 0 lr_mult 0.4444444444444444 loss 2.40

prediction:
 [1.89280792e-04 2.64807368e-05 1.15628034e-04 1.72795029e-04
 2.56835901e-05 1.44770456e-05 1.30431972e-05 1.30727800e-04
 2.71289470e-03 1.32313941e-03 2.29169382e-03 9.00231680e-06
 2.72542875e-05 1.37977477e-04 1.18895594e-04 4.04277962e-04
 3.52460444e-01 5.68393152e-04 7.92375416e-04 6.33973956e-01
 3.87270498e-04 5.19718633e-05 3.46783723e-04 1.51824370e-05
 5.31088563e-06 4.69901599e-04 9.74706723e-04 1.87986100e-03
 7.67413439e-05 3.48931712e-06 8.50204924e-06 5.59543059e-06
 1.62066819e-04 5.14766143e-05 8.11944574e-06 4.48189130e-05] 
 0.84446234
p [[0.00250313 0.00125156 0.00125156 0.00125156 0.00125156 0.00250313]
 [0.00125156 0.         0.04005006 0.00500626 0.00625782 0.00250313]
 [0.01126408 0.41301627 0.         0.         0.10638298 0.00250313]
 [0.00500626 0.15018773 0.00125156 0.00125156 0.19899875 0.01251564]
 [0.00125156 0.00250313 0.01126408 0.00625782 0.00125156 0.00125156]
 [0.00125156 0.00125156 0.00125156 0.00250313 0.00125156 0.00125156]]
move 13

game 52 completed in 9.291576147079468 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.362565 entropy 1.857205
training 1 lr_mult 0.4444444444444444 loss 2.3443906 entropy 1.8506943
training 2 lr_mult 0.4444444444444444 loss 2.3173208 entropy 1.8419788
training 3 lr_mult 0.4444444444444444 loss 2.2894666 entropy 1.8329871
training 4 lr_mult 0.4444444444444444 loss 2.2631218 entropy 1.8249424
kl 0.027903572
completed in 0.17203474044799805 s
game 53 completed in 9.10146713256836 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.3557122 entropy 1.8456764
training 1 lr_mult 0.4444444444444444 loss 2.332558 entropy 1.845968
training 2 lr_mult 0.4444444444444444 loss 2.3032708 entropy 1.8510399
training 3 lr_mult 0.4444444444444444 loss 2.2739837 entropy 1.8592855
training 4 lr_mult 0.4444444444444444 loss 2.243502 entropy 1.8685122
kl 0.024318121
completed in 0.16961216926574707 s
game 54 completed in 11.787399053573608 s 14 steps
training 0 lr_mult 0.4444444444444444 loss 2.

...

prediction:
 [2.25349795e-04 1.14554495e-01 8.98737833e-03 4.77027334e-03
 5.48483280e-04 6.46926754e-04 1.69350132e-02 6.26718625e-04
 2.25137789e-02 3.48851737e-03 2.55428348e-02 2.29250058e-03
 2.04003672e-03 1.04642451e-01 2.01544477e-04 4.14068066e-03
 3.19041945e-02 4.77991393e-03 5.82540967e-03 6.11926988e-02
 3.69149004e-03 1.73244090e-03 2.84343690e-01 1.38626536e-02
 6.10682298e-04 7.12536927e-03 3.08882957e-03 2.31762566e-02
 1.30110141e-03 1.44507485e-02 2.07780628e-03 1.04111596e-03
 1.37572666e-03 7.98357464e-03 2.18083948e-01 1.95516375e-04] 
 0.98297906
p [[0.00125156 0.03379224 0.00625782 0.00125156 0.00125156 0.00125156]
 [0.01251564 0.00125156 0.0212766  0.00500626 0.01126408 0.00750939]
 [0.00125156 0.00500626 0.00125156 0.61076345 0.         0.01877347]
 [0.02252816 0.02252816 0.         0.         0.         0.00125156]
 [0.00876095 0.02002503 0.         0.         0.         0.00625782]
 [0.00125156 0.00125156 0.00125156 0.01376721 0.15894869 0.00125156]]
move 15

game 257 completed in 9.210858821868896 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.4639812 entropy 1.8930844
training 1 lr_mult 0.4444444444444444 loss 2.4484403 entropy 1.8952324
training 2 lr_mult 0.4444444444444444 loss 2.426618 entropy 1.897779
training 3 lr_mult 0.4444444444444444 loss 2.4010007 entropy 1.9003916
training 4 lr_mult 0.4444444444444444 loss 2.3746955 entropy 1.9021585
kl 0.012792254
completed in 0.19163894653320312 s
game 258 completed in 9.322349071502686 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.4523716 entropy 1.9281912
training 1 lr_mult 0.4444444444444444 loss 2.424702 entropy 1.9258177
training 2 lr_mult 0.4444444444444444 loss 2.4026182 entropy 1.9218292
training 3 lr_mult 0.4444444444444444 loss 2.379251 entropy 1.9180472
training 4 lr_mult 0.4444444444444444 loss 2.3529227 entropy 1.9149079
kl 0.02021838
completed in 0.20804286003112793 s
game 259 completed in 9.071393966674805 s 11 steps
training 0 lr_mult 0.4444444444444444 loss

game 276 completed in 10.803669929504395 s 13 steps
training 0 lr_mult 0.4444444444444444 loss 2.4715466 entropy 1.8970623
training 1 lr_mult 0.4444444444444444 loss 2.4517183 entropy 1.8958185
training 2 lr_mult 0.4444444444444444 loss 2.4201994 entropy 1.8911644
training 3 lr_mult 0.4444444444444444 loss 2.3873613 entropy 1.8846538
training 4 lr_mult 0.4444444444444444 loss 2.3546777 entropy 1.8772087
kl 0.019499797
completed in 0.20764422416687012 s
game 277 completed in 7.621458292007446 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.3998666 entropy 1.8471079
training 1 lr_mult 0.4444444444444444 loss 2.3838642 entropy 1.8421665
training 2 lr_mult 0.4444444444444444 loss 2.3578663 entropy 1.8381587
training 3 lr_mult 0.4444444444444444 loss 2.3308885 entropy 1.8357493
training 4 lr_mult 0.4444444444444444 loss 2.3054028 entropy 1.8363016
kl 0.017950848
completed in 0.1703200340270996 s
game 278 completed in 6.00111985206604 s 7 steps
training 0 lr_mult 0.4444444444444444 lo

training 4 lr_mult 0.4444444444444444 loss 2.32333 entropy 1.8432133
kl 0.024412341
completed in 0.20113205909729004 s
game 295 completed in 13.190146923065186 s 16 steps
training 0 lr_mult 0.4444444444444444 loss 2.4569876 entropy 1.8671787
training 1 lr_mult 0.4444444444444444 loss 2.4419177 entropy 1.8630499
training 2 lr_mult 0.4444444444444444 loss 2.4162273 entropy 1.8564111
training 3 lr_mult 0.4444444444444444 loss 2.3880744 entropy 1.8485212
training 4 lr_mult 0.4444444444444444 loss 2.3595655 entropy 1.8404735
kl 0.0191673
completed in 0.2012770175933838 s
game 296 completed in 6.8316240310668945 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.5575163 entropy 1.8310341
training 1 lr_mult 0.4444444444444444 loss 2.5341997 entropy 1.8268018
training 2 lr_mult 0.4444444444444444 loss 2.4972212 entropy 1.8234122
training 3 lr_mult 0.4444444444444444 loss 2.458449 entropy 1.821903
training 4 lr_mult 0.4444444444444444 loss 2.42237 entropy 1.8219745
kl 0.014501032
completed 

prediction:
 [1.0758243e-03 5.4735960e-03 1.0322088e-03 6.0967766e-02 1.0882416e-02
 7.2510459e-04 8.4332200e-03 9.7219162e-02 8.9817873e-04 6.7589795e-03
 4.9871486e-04 4.9077352e-03 8.9139811e-04 2.6691522e-04 7.4097770e-05
 9.9231242e-05 1.2880175e-01 4.4455379e-02 4.5436901e-01 8.3343960e-02
 1.6971957e-04 1.8535001e-04 1.5473951e-04 1.1863294e-03 8.6028110e-03
 2.5718973e-04 2.2215461e-03 1.2317664e-03 3.3990335e-02 9.1852313e-03
 8.2927733e-04 1.0225161e-02 1.3469656e-02 1.8843393e-03 3.9471560e-03
 1.2849036e-03] 
 -0.85609984
p [[1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 5.00625782e-03 1.25156446e-13
  0.00000000e+00 1.25156446e-13]
 [1.25156446e-13 0.00000000e+00 1.25156446e-13 1.25156446e-13
  7.63454318e-02 1.25156446e-13]
 [1.25156446e-13 9.18648310e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 0.00000000e+00
  1.25156446e-13

In [83]:
k.save('n2-1500.model')

In [58]:
k = Train()

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:18: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:22: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:26: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:31: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. P

In [59]:
k.run(n_games=1)

2022-01-20 15:18:53.298288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


game 0 completed in 13.68191909790039 s 17 steps
training pipeline completed in 13.682025909423828 s


In [70]:
k.pv_net.predict(b.get_state().reshape(-1, 4, 6, 6))
k.save('test.model')

(array([0.02874902, 0.02816677, 0.02737665, 0.02786939, 0.02817374,
        0.02880505, 0.02842465, 0.02777849, 0.0273738 , 0.02997385,
        0.02888173, 0.0272564 , 0.02807645, 0.02801098, 0.02532525,
        0.02746927, 0.02689827, 0.02734488, 0.02804661, 0.02762723,
        0.02768766, 0.02771193, 0.02898358, 0.02825021, 0.02702713,
        0.02528107, 0.02628838, 0.02733569, 0.02815543, 0.02581511,
        0.027064  , 0.02844283, 0.02840897, 0.02999027, 0.0269438 ,
        0.02898561], dtype=float32),
 -0.029936753)

2022-01-20 15:28:35.683306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [71]:
p = PVNet2(6, 6)
p.restore('test.model')
p.predict(b.get_state().reshape(-1, 4, 6, 6))

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:18: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:22: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:26: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:31: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. P

INFO:tensorflow:Restoring parameters from test.model


2022-01-20 15:28:44.524546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-20 15:28:44.548772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


(array([0.02874902, 0.02816677, 0.02737665, 0.02786939, 0.02817374,
        0.02880505, 0.02842465, 0.02777849, 0.0273738 , 0.02997385,
        0.02888173, 0.0272564 , 0.02807645, 0.02801098, 0.02532525,
        0.02746927, 0.02689827, 0.02734488, 0.02804661, 0.02762723,
        0.02768766, 0.02771193, 0.02898358, 0.02825021, 0.02702713,
        0.02528107, 0.02628838, 0.02733569, 0.02815543, 0.02581511,
        0.027064  , 0.02844283, 0.02840897, 0.02999027, 0.0269438 ,
        0.02898561], dtype=float32),
 -0.029936753)

In [38]:
model = Train()
model.run()

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:17: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:21: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:25: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:30: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. P

game 0 completed in 10.20585298538208 s 12 steps
game 1 completed in 12.43356990814209 s 15 steps
game 2 completed in 12.412672996520996 s 15 steps
game 3 completed in 16.40099811553955 s 20 steps
game 4 completed in 8.33475923538208 s 10 steps


2022-01-20 12:10:51.105840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


training 0 lr_mult 1.0 loss 4.6394787 entropy 3.581136
training 1 lr_mult 1.0 loss 4.628387 entropy 3.5826116
training 2 lr_mult 1.0 loss 4.617593 entropy 3.5819411
training 3 lr_mult 1.0 loss 4.5853596 entropy 3.5821579
training 4 lr_mult 1.0 loss 4.5643735 entropy 3.5815341
kl 0.0034765895
completed in 0.5984289646148682 s
game 5 completed in 10.418401956558228 s 12 steps
training 0 lr_mult 1.5 loss 4.520674 entropy 3.5799809
training 1 lr_mult 1.5 loss 4.445717 entropy 3.573915
training 2 lr_mult 1.5 loss 4.3892646 entropy 3.5645156
training 3 lr_mult 1.5 loss 4.3492365 entropy 3.5629597
training 4 lr_mult 1.5 loss 4.2846427 entropy 3.5693307
kl 0.004093402
completed in 0.303513765335083 s
game 6 completed in 13.651462078094482 s 16 steps
training 0 lr_mult 2.25 loss 4.2197266 entropy 3.5744061
training 1 lr_mult 2.25 loss 4.191503 entropy 3.5744042
training 2 lr_mult 2.25 loss 4.2177305 entropy 3.5664349
training 3 lr_mult 2.25 loss 4.011654 entropy 3.553117
training 4 lr_mult 2.25

game 26 completed in 12.000976800918579 s 13 steps
training 0 lr_mult 1.0 loss 3.8849018 entropy 3.2332413
training 1 lr_mult 1.0 loss 3.8604217 entropy 3.2617896
training 2 lr_mult 1.0 loss 3.8180218 entropy 3.2823243
training 3 lr_mult 1.0 loss 3.7674122 entropy 3.2843022
training 4 lr_mult 1.0 loss 3.7299535 entropy 3.269187
kl 0.043320514
completed in 0.2742042541503906 s
game 27 completed in 7.415899753570557 s 8 steps
training 0 lr_mult 0.6666666666666666 loss 3.793778 entropy 3.25625
training 1 lr_mult 0.6666666666666666 loss 3.776403 entropy 3.2562332
training 2 lr_mult 0.6666666666666666 loss 3.745338 entropy 3.2656827
training 3 lr_mult 0.6666666666666666 loss 3.71293 entropy 3.274415
training 4 lr_mult 0.6666666666666666 loss 3.6853256 entropy 3.277104
kl 0.03681104
completed in 0.2846081256866455 s
game 28 completed in 18.898313999176025 s 21 steps
training 0 lr_mult 0.6666666666666666 loss 3.9653125 entropy 3.2783716
training 1 lr_mult 0.6666666666666666 loss 3.9416149 ent

...

prediction:
 [0.00059193 0.00394273 0.0016246  0.00422478 0.01603831 0.000905
 0.00825699 0.01348106 0.06255358 0.01355516 0.04043624 0.01227196
 0.00227353 0.09260082 0.08728167 0.06348974 0.07411592 0.00158073
 0.00169115 0.05847909 0.07786875 0.08415283 0.08277591 0.00300291
 0.00876257 0.04274169 0.01199691 0.0666776  0.03192552 0.00529121
 0.00085585 0.01236108 0.00281583 0.00207748 0.00673533 0.00056379] 
 0.41505298
p [[0.00125156 0.00125156 0.00375469 0.00375469 0.00125156 0.00125156]
 [0.00125156 0.02878598 0.0350438  0.00500626 0.00500626 0.00125156]
 [0.00375469 0.10638298 0.08135169 0.08010013 0.05006258 0.00250313]
 [0.00125156 0.18147685 0.05757196 0.10387985 0.02377972 0.10387985]
 [0.00125156 0.02002503 0.         0.02753442 0.0563204  0.00250313]
 [0.00125156 0.00125156 0.00125156 0.00125156 0.00125156 0.00125156]]
move 19
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0. 

training 3 lr_mult 0.4444444444444444 loss 3.10256 entropy 2.6734843
training 4 lr_mult 0.4444444444444444 loss 3.0699105 entropy 2.6738486
kl 0.009282004
completed in 0.3315761089324951 s
game 253 completed in 19.286153078079224 s 21 steps
training 0 lr_mult 0.6666666666666666 loss 3.224401 entropy 2.6726618
training 1 lr_mult 0.6666666666666666 loss 3.19512 entropy 2.6621885
training 2 lr_mult 0.6666666666666666 loss 3.1543655 entropy 2.6516995
training 3 lr_mult 0.6666666666666666 loss 3.1048498 entropy 2.6515498
training 4 lr_mult 0.6666666666666666 loss 3.054365 entropy 2.661375
kl 0.015595285
completed in 0.28951311111450195 s
game 254 completed in 14.722854137420654 s 16 steps
training 0 lr_mult 0.6666666666666666 loss 3.190583 entropy 2.6690278
training 1 lr_mult 0.6666666666666666 loss 3.1667626 entropy 2.6790137
training 2 lr_mult 0.6666666666666666 loss 3.124983 entropy 2.6780753
training 3 lr_mult 0.6666666666666666 loss 3.0819545 entropy 2.6661339
training 4 lr_mult 0.6666

game 271 completed in 6.572465896606445 s 7 steps
training 0 lr_mult 0.6666666666666666 loss 3.188706 entropy 2.5599248
training 1 lr_mult 0.6666666666666666 loss 3.1494443 entropy 2.5548124
training 2 lr_mult 0.6666666666666666 loss 3.0934021 entropy 2.5609226
training 3 lr_mult 0.6666666666666666 loss 3.0446029 entropy 2.570363
training 4 lr_mult 0.6666666666666666 loss 3.003855 entropy 2.5752096
kl 0.02284014
completed in 0.3030421733856201 s
game 272 completed in 11.120978116989136 s 12 steps
training 0 lr_mult 0.6666666666666666 loss 3.1903934 entropy 2.5666924
training 1 lr_mult 0.6666666666666666 loss 3.1616688 entropy 2.5567474
training 2 lr_mult 0.6666666666666666 loss 3.1136904 entropy 2.5543423
training 3 lr_mult 0.6666666666666666 loss 3.0617514 entropy 2.5654974
training 4 lr_mult 0.6666666666666666 loss 3.0136318 entropy 2.586688
kl 0.034572646
completed in 0.34215211868286133 s
game 273 completed in 10.692086935043335 s 11 steps
training 0 lr_mult 0.6666666666666666 loss

training 2 lr_mult 0.6666666666666666 loss 3.0597584 entropy 2.532612
training 3 lr_mult 0.6666666666666666 loss 3.011808 entropy 2.5201056
training 4 lr_mult 0.6666666666666666 loss 2.969052 entropy 2.512454
kl 0.020643812
completed in 0.32325291633605957 s
game 290 completed in 7.2246458530426025 s 8 steps
training 0 lr_mult 0.6666666666666666 loss 3.2213705 entropy 2.545236
training 1 lr_mult 0.6666666666666666 loss 3.1821232 entropy 2.5621605
training 2 lr_mult 0.6666666666666666 loss 3.1238086 entropy 2.5809884
training 3 lr_mult 0.6666666666666666 loss 3.0704877 entropy 2.5911913
training 4 lr_mult 0.6666666666666666 loss 3.0194185 entropy 2.5886805
kl 0.034671254
completed in 0.312028169631958 s
game 291 completed in 12.95711064338684 s 14 steps
training 0 lr_mult 0.6666666666666666 loss 3.104727 entropy 2.5651278
training 1 lr_mult 0.6666666666666666 loss 3.0826511 entropy 2.5557833
training 2 lr_mult 0.6666666666666666 loss 3.0423548 entropy 2.5544662
training 3 lr_mult 0.6666

prediction:
 [0.01173904 0.00755928 0.03267944 0.01004538 0.00469016 0.0048984
 0.00871112 0.0097214  0.03833952 0.35555324 0.01558099 0.00354991
 0.00329809 0.02689761 0.00584402 0.00214134 0.02816268 0.00729666
 0.00292067 0.03780636 0.00323558 0.00468816 0.02818508 0.00581669
 0.01056017 0.01812175 0.21056448 0.01892925 0.01267787 0.01148121
 0.00525931 0.00202252 0.02092758 0.01035063 0.00947593 0.01026864] 
 0.30034667
p [[1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 2.50312891e-03 2.25281602e-02 3.25406758e-02
  1.25156446e-03 1.25156446e-03]
 [1.25156446e-13 3.75469337e-03 0.00000000e+00 0.00000000e+00
  2.50312891e-03 1.25156446e-13]
 [1.25156446e-13 1.25156446e-03 0.00000000e+00 0.00000000e+00
  1.25156446e-03 1.25156446e-13]
 [1.25156446e-13 1.25156446e-03 1.12640801e-02 9.14893617e-01
  2.50312891e-03 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-03
  1.25156446e-13 1.25156446e-13]]

In [39]:
model.run()


game 0 completed in 9.026176929473877 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 3.094229 entropy 2.4553971
training 1 lr_mult 0.4444444444444444 loss 3.0804331 entropy 2.461153
training 2 lr_mult 0.4444444444444444 loss 3.0460992 entropy 2.468775
training 3 lr_mult 0.4444444444444444 loss 3.0024881 entropy 2.4735408
training 4 lr_mult 0.4444444444444444 loss 2.96221 entropy 2.4722147
kl 0.019545441
completed in 0.29179811477661133 s
game 1 completed in 6.601289987564087 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 3.0083284 entropy 2.4812946
training 1 lr_mult 0.4444444444444444 loss 2.9982338 entropy 2.472268
training 2 lr_mult 0.4444444444444444 loss 2.9779449 entropy 2.4659348
training 3 lr_mult 0.4444444444444444 loss 2.9491947 entropy 2.4639454
training 4 lr_mult 0.4444444444444444 loss 2.9149377 entropy 2.4665737
kl 0.012923163
completed in 0.30759406089782715 s
game 2 completed in 12.0640230178833 s 13 steps
training 0 lr_mult 0.4444444444444444 loss 3.067191

training 2 lr_mult 0.6666666666666666 loss 2.904401 entropy 2.4090385
training 3 lr_mult 0.6666666666666666 loss 2.8553898 entropy 2.4068408
training 4 lr_mult 0.6666666666666666 loss 2.8143163 entropy 2.4099774
kl 0.025351774
completed in 0.32304883003234863 s
game 19 completed in 13.030799150466919 s 14 steps
training 0 lr_mult 0.6666666666666666 loss 2.9384475 entropy 2.4159372
training 1 lr_mult 0.6666666666666666 loss 2.9071538 entropy 2.4348125
training 2 lr_mult 0.6666666666666666 loss 2.866565 entropy 2.4546928
training 3 lr_mult 0.6666666666666666 loss 2.8321247 entropy 2.4684856
training 4 lr_mult 0.6666666666666666 loss 2.8054445 entropy 2.4753957
kl 0.023606826
completed in 0.32593798637390137 s
game 20 completed in 8.218488693237305 s 9 steps
training 0 lr_mult 0.6666666666666666 loss 3.077183 entropy 2.5422583
training 1 lr_mult 0.6666666666666666 loss 3.052214 entropy 2.5400295
training 2 lr_mult 0.6666666666666666 loss 3.0107024 entropy 2.537404
training 3 lr_mult 0.666

training 3 lr_mult 0.6666666666666666 loss 2.8087015 entropy 2.376173
training 4 lr_mult 0.6666666666666666 loss 2.7660782 entropy 2.386223
kl 0.0249204
completed in 0.28464317321777344 s
game 37 completed in 10.432674884796143 s 11 steps
training 0 lr_mult 0.6666666666666666 loss 2.94059 entropy 2.4478843
training 1 lr_mult 0.6666666666666666 loss 2.9181545 entropy 2.452827
training 2 lr_mult 0.6666666666666666 loss 2.8792448 entropy 2.4509876
training 3 lr_mult 0.6666666666666666 loss 2.8375323 entropy 2.4425857
training 4 lr_mult 0.6666666666666666 loss 2.796828 entropy 2.4298775
kl 0.022993855
completed in 0.33160996437072754 s
game 38 completed in 6.680757999420166 s 7 steps
training 0 lr_mult 0.6666666666666666 loss 2.9082592 entropy 2.4169016
training 1 lr_mult 0.6666666666666666 loss 2.8817656 entropy 2.4050043
training 2 lr_mult 0.6666666666666666 loss 2.838505 entropy 2.3943005
training 3 lr_mult 0.6666666666666666 loss 2.7935636 entropy 2.385439
training 4 lr_mult 0.66666666

prediction:
 [0.00336446 0.00172487 0.00349357 0.0017654  0.03146284 0.00018638
 0.00159049 0.06387119 0.04245533 0.11335321 0.00871331 0.05219903
 0.00571777 0.03208628 0.00437505 0.10672527 0.00785058 0.00362661
 0.00367008 0.00944979 0.0723919  0.00899553 0.07643837 0.00528296
 0.07525904 0.00836452 0.04736133 0.04808377 0.11747164 0.00091022
 0.0001824  0.02205388 0.00583649 0.00882554 0.00255826 0.00230284] 
 0.9996216
p [[0.00125156 0.00125156 0.00125156 0.00125156 0.00125156 0.00125156]
 [0.00125156 0.00375469 0.01501877 0.72340426 0.00125156 0.00125156]
 [0.00125156 0.01376721 0.         0.07509387 0.00250313 0.00125156]
 [0.00125156 0.         0.02503129 0.         0.00500626 0.00125156]
 [0.00125156 0.00125156 0.0738423  0.02878598 0.00625782 0.00125156]
 [0.00125156 0.00125156 0.00125156 0.00125156 0.00125156 0.00125156]]
move 9
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0. 

game 52 completed in 7.636742115020752 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.9286628 entropy 2.3562784
training 1 lr_mult 0.4444444444444444 loss 2.911796 entropy 2.3545432
training 2 lr_mult 0.4444444444444444 loss 2.8772652 entropy 2.3548484
training 3 lr_mult 0.4444444444444444 loss 2.837936 entropy 2.3571613
training 4 lr_mult 0.4444444444444444 loss 2.8007948 entropy 2.3602736
kl 0.01388003
completed in 0.3223140239715576 s
game 53 completed in 6.78870701789856 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.8105145 entropy 2.3254688
training 1 lr_mult 0.4444444444444444 loss 2.7837477 entropy 2.3301353
training 2 lr_mult 0.4444444444444444 loss 2.7471964 entropy 2.334608
training 3 lr_mult 0.4444444444444444 loss 2.7077968 entropy 2.3352358
training 4 lr_mult 0.4444444444444444 loss 2.6711268 entropy 2.33072
kl 0.018993214
completed in 0.27364301681518555 s
game 54 completed in 10.300014019012451 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.8067

...

prediction:
 [4.1235802e-03 4.7215555e-02 1.1148435e-02 6.3070590e-03 2.3364323e-01
 3.6224984e-02 5.1956702e-02 1.5888344e-02 8.6930608e-03 3.9535942e-03
 2.1664632e-02 5.4161273e-02 2.6121123e-03 2.0595477e-03 3.8064318e-05
 2.1854715e-04 3.7565929e-04 3.7717089e-02 9.8224565e-02 1.5507810e-04
 5.3355761e-04 2.9231107e-05 1.1371489e-03 1.7666448e-02 1.5024746e-02
 2.0842036e-02 3.5565351e-03 4.9929237e-03 3.9347403e-02 3.5943154e-02
 2.5159776e-02 1.4528312e-01 3.7140315e-03 4.5354213e-03 4.3345921e-02
 2.5076659e-03] 
 -0.9279361
p [[0.01877347 0.04255319 0.02753442 0.03379224 0.0350438  0.00750939]
 [0.0350438  0.23529412 0.02377972 0.01001252 0.02753442 0.03128911]
 [0.0387985  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.03254068]
 [0.02753442 0.         0.00625782 0.         0.13892365 0.05381727]
 [0.02628285 0.02252816 0.02503129 0.03254068 0.04755945 0.02002503]]
move 7
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 

game 262 completed in 10.265758037567139 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.5964773 entropy 1.9317849
training 1 lr_mult 0.4444444444444444 loss 2.5819833 entropy 1.933647
training 2 lr_mult 0.4444444444444444 loss 2.5502264 entropy 1.9402487
training 3 lr_mult 0.4444444444444444 loss 2.5111625 entropy 1.947894
training 4 lr_mult 0.4444444444444444 loss 2.4733603 entropy 1.9531326
kl 0.02360173
completed in 0.28708600997924805 s
game 263 completed in 6.717137813568115 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.520391 entropy 1.9646741
training 1 lr_mult 0.4444444444444444 loss 2.5016775 entropy 1.9592588
training 2 lr_mult 0.4444444444444444 loss 2.4751496 entropy 1.951565
training 3 lr_mult 0.4444444444444444 loss 2.4483438 entropy 1.9456602
training 4 lr_mult 0.4444444444444444 loss 2.4208696 entropy 1.9429346
kl 0.024529796
completed in 0.31358909606933594 s
game 264 completed in 6.501478910446167 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 

training 2 lr_mult 0.4444444444444444 loss 2.5148587 entropy 1.9250381
training 3 lr_mult 0.4444444444444444 loss 2.4815643 entropy 1.9390936
training 4 lr_mult 0.4444444444444444 loss 2.444028 entropy 1.9461422
kl 0.025131822
completed in 0.330111026763916 s
game 281 completed in 8.417089939117432 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.6470003 entropy 1.9551235
training 1 lr_mult 0.4444444444444444 loss 2.6250541 entropy 1.9463574
training 2 lr_mult 0.4444444444444444 loss 2.5948584 entropy 1.9345934
training 3 lr_mult 0.4444444444444444 loss 2.5636263 entropy 1.9265845
training 4 lr_mult 0.4444444444444444 loss 2.5307744 entropy 1.9257863
kl 0.027513687
completed in 0.31581902503967285 s
game 282 completed in 6.72403883934021 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.631058 entropy 1.9742227
training 1 lr_mult 0.4444444444444444 loss 2.6131876 entropy 1.9896891
training 2 lr_mult 0.4444444444444444 loss 2.585925 entropy 2.0051908
training 3 lr_mult 0.4444

training 3 lr_mult 0.4444444444444444 loss 2.476453 entropy 1.9104099
training 4 lr_mult 0.4444444444444444 loss 2.4392602 entropy 1.9179637
kl 0.01990429
completed in 0.27364301681518555 s
prediction:
 [0.00106805 0.00218956 0.00620073 0.00477872 0.00234696 0.00336887
 0.00293443 0.002444   0.01052963 0.06382839 0.02693655 0.00221701
 0.00335807 0.01417227 0.06538039 0.20242853 0.04644618 0.00602467
 0.00319874 0.07000396 0.21272123 0.07139545 0.01406004 0.00291625
 0.00214043 0.03015892 0.084039   0.01470123 0.00473528 0.00263422
 0.00261427 0.00126762 0.00887795 0.00406601 0.0032845  0.00053198] 
 -0.39416352
p [[1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 2.50312891e-03 1.25156446e-03
  1.25156446e-13 1.25156446e-13]
 [1.25156446e-13 1.25156446e-03 2.44055069e-01 4.25531915e-01
  1.25156446e-03 1.25156446e-13]
 [1.25156446e-13 1.25156446e-03 1.31414268e-01 1.82728411e-01
  3.75469337e-03 1.25156446e-13

prediction:
 [1.2086938e-03 1.3967523e-02 6.6907294e-03 4.1162875e-03 1.2771874e-02
 1.5664246e-03 3.3709195e-03 2.4165815e-02 7.0090028e-03 9.3079843e-02
 8.9655314e-03 7.3705669e-03 7.7586882e-03 1.4945895e-02 1.9765395e-01
 1.0633184e-04 2.0706494e-04 1.0758005e-01 2.3204304e-02 2.2622079e-04
 1.2776855e-04 8.6299367e-02 6.8184711e-02 6.9506755e-03 1.6624575e-02
 1.0510747e-02 1.6986054e-01 4.7703847e-02 2.2918196e-02 7.8141950e-03
 1.6669559e-03 4.4783326e-03 5.3234897e-03 8.6028567e-03 5.9785489e-03
 9.8977715e-04] 
 0.91726077
p [[0.00125156 0.0563204  0.0212766  0.01126408 0.00500626 0.00125156]
 [0.02753442 0.00250313 0.04505632 0.00625782 0.11138924 0.02628285]
 [0.00125156 0.         0.         0.         0.         0.        ]
 [0.01251564 0.33416771 0.         0.09261577 0.00750939 0.00375469]
 [0.02252816 0.10638298 0.02002503 0.         0.01126408 0.03754693]
 [0.00125156 0.00250313 0.01126408 0.00375469 0.01501877 0.00125156]]
move 19
board
 [[ 0.  0.  0.  0.  0.  0.]
 [

In [40]:
model.save('600.model')

2022-01-20 14:09:38.688053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [41]:
model.run()

game 0 completed in 10.095508813858032 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.513793 entropy 1.9343067
training 1 lr_mult 0.4444444444444444 loss 2.4938896 entropy 1.9411106
training 2 lr_mult 0.4444444444444444 loss 2.4640594 entropy 1.9489235
training 3 lr_mult 0.4444444444444444 loss 2.433223 entropy 1.9527619
training 4 lr_mult 0.4444444444444444 loss 2.4015727 entropy 1.9493003
kl 0.026626127
completed in 0.2795288562774658 s
game 1 completed in 6.527832746505737 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.5549695 entropy 1.9087353
training 1 lr_mult 0.4444444444444444 loss 2.5358918 entropy 1.9055998
training 2 lr_mult 0.4444444444444444 loss 2.5033984 entropy 1.9065278
training 3 lr_mult 0.4444444444444444 loss 2.4674277 entropy 1.9117503
training 4 lr_mult 0.4444444444444444 loss 2.4352825 entropy 1.9200323
kl 0.020961892
completed in 0.2685718536376953 s
game 2 completed in 8.347821950912476 s 9 steps
training 0 lr_mult 0.4444444444444444 loss 2.588

training 1 lr_mult 0.4444444444444444 loss 2.4822779 entropy 1.9162962
training 2 lr_mult 0.4444444444444444 loss 2.4542358 entropy 1.9106786
training 3 lr_mult 0.4444444444444444 loss 2.420704 entropy 1.9041018
training 4 lr_mult 0.4444444444444444 loss 2.3856668 entropy 1.899416
kl 0.01482099
completed in 0.3476710319519043 s
game 19 completed in 13.581148147583008 s 15 steps
training 0 lr_mult 0.4444444444444444 loss 2.5000517 entropy 1.9122177
training 1 lr_mult 0.4444444444444444 loss 2.4778726 entropy 1.9113041
training 2 lr_mult 0.4444444444444444 loss 2.4429116 entropy 1.9079008
training 3 lr_mult 0.4444444444444444 loss 2.4042156 entropy 1.9037268
training 4 lr_mult 0.4444444444444444 loss 2.3681808 entropy 1.9007993
kl 0.023973484
completed in 0.3275270462036133 s
game 20 completed in 6.566236257553101 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.4889147 entropy 1.9249933
training 1 lr_mult 0.4444444444444444 loss 2.4669113 entropy 1.9382288
training 2 lr_mult 0.444

training 3 lr_mult 0.4444444444444444 loss 2.3954368 entropy 1.9487563
training 4 lr_mult 0.4444444444444444 loss 2.3623896 entropy 1.9474087
kl 0.017543662
completed in 0.26756882667541504 s
game 37 completed in 17.00275206565857 s 19 steps
training 0 lr_mult 0.4444444444444444 loss 2.453549 entropy 1.9041284
training 1 lr_mult 0.4444444444444444 loss 2.4373877 entropy 1.8993499
training 2 lr_mult 0.4444444444444444 loss 2.4101129 entropy 1.8947692
training 3 lr_mult 0.4444444444444444 loss 2.3765197 entropy 1.891593
training 4 lr_mult 0.4444444444444444 loss 2.3422534 entropy 1.8904054
kl 0.015505479
completed in 0.32628297805786133 s
game 38 completed in 16.8403217792511 s 19 steps
training 0 lr_mult 0.4444444444444444 loss 2.532041 entropy 1.9073291
training 1 lr_mult 0.4444444444444444 loss 2.503185 entropy 1.910405
training 2 lr_mult 0.4444444444444444 loss 2.4592497 entropy 1.915062
training 3 lr_mult 0.4444444444444444 loss 2.4166634 entropy 1.9211853
training 4 lr_mult 0.44444

prediction:
 [2.8015402e-04 1.4883054e-04 1.1833881e-03 6.0885807e-04 1.4196811e-04
 3.4925350e-05 1.5226670e-04 3.0363805e-03 2.3222068e-01 3.0218518e-01
 4.3768496e-03 6.5490407e-05 4.2538819e-04 1.5560017e-04 3.3831496e-03
 9.3999325e-04 3.9943121e-04 2.0932101e-04 1.1090873e-04 3.9829081e-04
 3.8292152e-03 1.0199288e-03 6.1002451e-05 2.9715779e-04 1.3165714e-04
 1.7288313e-03 3.0270958e-01 1.2866928e-01 8.1780609e-03 9.1109745e-05
 6.0142727e-05 1.5221386e-04 1.1150157e-03 9.7796798e-04 1.2833679e-04
 3.9353422e-04] 
 -0.44149327
p [[0.00125156 0.00125156 0.01126408 0.00125156 0.00125156 0.00125156]
 [0.00125156 0.00876095 0.11013767 0.03254068 0.1339174  0.00125156]
 [0.00125156 0.00125156 0.00250313 0.         0.29662078 0.00876095]
 [0.00125156 0.04505632 0.00876095 0.         0.         0.00125156]
 [0.00125156 0.00250313 0.02628285 0.14142678 0.12640801 0.00125156]
 [0.00125156 0.00125156 0.00750939 0.01627034 0.00125156 0.00125156]]
move 16
board
 [[ 0.  0.  0.  0.  0.  0.]
 

game 57 completed in 7.4498131275177 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.4376776 entropy 1.9193053
training 1 lr_mult 0.4444444444444444 loss 2.4189649 entropy 1.9161454
training 2 lr_mult 0.4444444444444444 loss 2.3898716 entropy 1.9116745
training 3 lr_mult 0.4444444444444444 loss 2.3559954 entropy 1.9067056
training 4 lr_mult 0.4444444444444444 loss 2.3226697 entropy 1.9020827
kl 0.016460646
completed in 0.31597018241882324 s
game 58 completed in 12.579529047012329 s 14 steps
training 0 lr_mult 0.4444444444444444 loss 2.5899847 entropy 1.930832
training 1 lr_mult 0.4444444444444444 loss 2.5639577 entropy 1.93309
training 2 lr_mult 0.4444444444444444 loss 2.5318067 entropy 1.938261
training 3 lr_mult 0.4444444444444444 loss 2.500868 entropy 1.9443688
training 4 lr_mult 0.4444444444444444 loss 2.4722676 entropy 1.9496164
kl 0.013998792
completed in 0.30773091316223145 s
game 59 completed in 11.632242918014526 s 13 steps
training 0 lr_mult 0.4444444444444444 loss 2.5

...

prediction:
 [0.00964776 0.20665705 0.00355887 0.02873594 0.02308655 0.00135135
 0.00991805 0.03368733 0.03610748 0.00378036 0.00334137 0.00505084
 0.00232631 0.00103974 0.00429826 0.00110015 0.00376923 0.09324216
 0.028661   0.01069242 0.00034927 0.01191724 0.00065624 0.00324038
 0.0057381  0.0035395  0.0098649  0.03782888 0.06661819 0.00878145
 0.00617913 0.02006702 0.13874432 0.00206268 0.16777919 0.00658137] 
 0.8633132
p [[1.25156446e-13 5.50688360e-02 1.25156446e-13 1.25156446e-13
  1.25156446e-13 1.25156446e-13]
 [0.00000000e+00 1.25156446e-13 0.00000000e+00 1.25156446e-13
  5.81977472e-01 1.25156446e-13]
 [1.25156446e-13 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.25156446e-13 1.25156446e-13 0.00000000e+00 1.25156446e-13
  0.00000000e+00 1.25156446e-13]
 [1.25156446e-13 2.12765957e-02 1.25156446e-13 1.25156446e-13
  0.00000000e+00 1.25156446e-13]
 [1.25156446e-13 1.25156446e-13 1.25156446e-13 1.25156446e-13
  3.41677096e-01 1.25156446e-13]]

training 3 lr_mult 0.4444444444444444 loss 2.53953 entropy 1.9228791
training 4 lr_mult 0.4444444444444444 loss 2.4984815 entropy 1.9312816
kl 0.019601978
completed in 0.315079927444458 s
game 265 completed in 10.281911134719849 s 12 steps
training 0 lr_mult 0.4444444444444444 loss 2.5863168 entropy 1.9154191
training 1 lr_mult 0.4444444444444444 loss 2.556981 entropy 1.9216721
training 2 lr_mult 0.4444444444444444 loss 2.5183082 entropy 1.9263182
training 3 lr_mult 0.4444444444444444 loss 2.480029 entropy 1.9302284
training 4 lr_mult 0.4444444444444444 loss 2.4379296 entropy 1.9341139
kl 0.020311346
completed in 0.3013641834259033 s
game 266 completed in 7.306581020355225 s 8 steps
training 0 lr_mult 0.4444444444444444 loss 2.5754197 entropy 1.9466939
training 1 lr_mult 0.4444444444444444 loss 2.5534055 entropy 1.9468771
training 2 lr_mult 0.4444444444444444 loss 2.5138705 entropy 1.9438449
training 3 lr_mult 0.4444444444444444 loss 2.4652803 entropy 1.9378266
training 4 lr_mult 0.444

game 283 completed in 6.27696681022644 s 7 steps
training 0 lr_mult 0.4444444444444444 loss 2.5352228 entropy 1.9352939
training 1 lr_mult 0.4444444444444444 loss 2.5128121 entropy 1.9387729
training 2 lr_mult 0.4444444444444444 loss 2.4808578 entropy 1.9410349
training 3 lr_mult 0.4444444444444444 loss 2.4459033 entropy 1.9406767
training 4 lr_mult 0.4444444444444444 loss 2.4105213 entropy 1.9375012
kl 0.01337751
completed in 0.2978780269622803 s
game 284 completed in 9.459536790847778 s 11 steps
training 0 lr_mult 0.4444444444444444 loss 2.585432 entropy 1.9444861
training 1 lr_mult 0.4444444444444444 loss 2.560864 entropy 1.9337171
training 2 lr_mult 0.4444444444444444 loss 2.5228136 entropy 1.9200335
training 3 lr_mult 0.4444444444444444 loss 2.4820824 entropy 1.9076436
training 4 lr_mult 0.4444444444444444 loss 2.442019 entropy 1.8983816
kl 0.016816754
completed in 0.35178589820861816 s
game 285 completed in 8.965224981307983 s 10 steps
training 0 lr_mult 0.4444444444444444 loss 2

prediction:
 [7.92010105e-05 2.23632785e-04 2.05750795e-04 3.09665164e-04
 2.07321995e-04 3.54435579e-05 1.44535050e-04 2.33662606e-04
 1.10223619e-02 9.57463123e-03 6.22412264e-02 1.84398639e-04
 2.63161538e-03 2.78942943e-01 1.39776230e-01 2.82882955e-02
 3.26328503e-04 1.07403175e-04 1.18387572e-04 1.62552707e-04
 7.60498717e-02 1.52958497e-01 1.72011659e-01 1.78557483e-03
 1.30432032e-04 4.11440805e-02 1.23492815e-02 7.40110083e-03
 1.42873498e-04 1.39760305e-04 4.21522454e-05 4.31203895e-04
 1.50711916e-04 1.29972977e-04 2.72210804e-04 4.51135202e-05] 
 0.19668975
p [[0.00125156 0.00250313 0.00125156 0.00250313 0.00125156 0.00125156]
 [0.00250313 0.00125156 0.14768461 0.07634543 0.00125156 0.00125156]
 [0.00125156 0.01126408 0.00375469 0.         0.30162703 0.00125156]
 [0.00375469 0.08385482 0.2290363  0.00375469 0.00750939 0.00125156]
 [0.00375469 0.00375469 0.08135169 0.00250313 0.00250313 0.00625782]
 [0.00125156 0.00250313 0.00250313 0.00250313 0.00125156 0.00125156]]
move 16

prediction:
 [1.1286080e-02 4.6176356e-03 1.4237895e-02 5.2516633e-01 4.0782723e-03
 2.9515108e-04 1.2693401e-02 2.4021523e-02 8.7599625e-04 9.1740303e-03
 1.7536843e-02 3.6905978e-02 6.8706772e-03 1.0182000e-03 5.7284045e-03
 5.9645704e-04 1.8873625e-04 6.7370408e-03 1.7093483e-03 3.4274111e-04
 3.0041841e-04 3.9468907e-02 1.6918298e-03 2.8018756e-03 1.7339194e-02
 1.8989664e-02 7.1880617e-03 6.8380550e-04 8.8223265e-03 4.4224687e-02
 2.7820247e-04 3.2792003e-03 1.4833458e-01 1.3608509e-02 6.6733230e-03
 2.2347302e-03] 
 0.9901453
p [[0.00125156 0.00125156 0.14768461 0.00125156 0.00125156 0.00125156]
 [0.00125156 0.03003755 0.         0.         0.04005006 0.0563204 ]
 [0.00125156 0.00125156 0.00125156 0.         0.         0.00125156]
 [0.00125156 0.00125156 0.         0.         0.         0.        ]
 [0.00876095 0.01376721 0.02377972 0.         0.00500626 0.00125156]
 [0.00125156 0.00625782 0.00125156 0.64705882 0.00125156 0.00125156]]
move 33
board
 [[ 0.  0.  0.  0.  0.  0.]
 [ 

In [53]:
pv = PVNet2(6, 6)
pv.restore('600.model')
b = Board(6, 4)
pv.predict(b.get_state().reshape(-1, 4, 6, 6))

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:18: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:22: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:26: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:31: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. P

INFO:tensorflow:Restoring parameters from 600.model


2022-01-20 15:17:25.138559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-20 15:17:25.166886: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: 2 root error(s) found.
  (0) INVALID_ARGUMENT: Input to reshape is a tensor with 72 values, but the requested shape requires a multiple of 144
	 [[node Reshape
 (defined at /var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:36)
]]
	 [[dense_2/Tanh/_105]]
  (1) INVALID_ARGUMENT: Input to reshape is a tensor with 72 values, but the requested shape requires a multiple of 144
	 [[node Reshape
 (defined at /var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/3538894304.py:36)
]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node Reshape:
In[0] conv2d_3/Relu (defined at /Users/fanrongqi/miniforge3/envs/tf/lib/python3.9/site-packages/keras/layers/convolutional.py:273)	
In[1] Reshape/shape:

Operation defined at: (most recent call last)
>>>   File "/Users/fanrongqi/miniforge3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_mai

In [65]:
model.pv_net.predict(b.get_state().reshape(-1, 4, 6, 6))


(array([2.3689392e-07, 3.1376342e-07, 5.0886496e-07, 5.1594026e-07,
        2.7964430e-07, 4.7143509e-08, 6.6497114e-07, 4.2883763e-05,
        3.7276754e-04, 3.1938957e-04, 4.5294495e-05, 2.6526584e-07,
        9.0949135e-07, 6.0766219e-04, 2.2480451e-01, 2.7523169e-01,
        4.1038700e-04, 1.2267570e-06, 5.6432191e-07, 3.1323172e-04,
        2.8378546e-01, 2.1240395e-01, 6.2836619e-04, 3.8217266e-07,
        1.8744321e-07, 3.6152513e-05, 3.4512402e-04, 6.0383789e-04,
        4.0508086e-05, 4.0719797e-07, 5.1515578e-08, 3.5078295e-07,
        4.2418924e-07, 6.6745918e-07, 7.9944306e-07, 1.5953768e-07],
       dtype=float32),
 0.3153984)

1

In [48]:
pv = PVNet2(6, 6)
pv.restore('900.model')

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:17: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv1 = tf.layers.conv2d(inputs=self.input_state,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:21: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:25: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:30: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. P

INFO:tensorflow:Restoring parameters from 900.model


2022-01-20 15:08:16.327502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-20 15:08:16.594658: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at save_restore_v2_ops.cc:207 : NOT_FOUND: Key batch_normalization/beta/Adam_10 not found in checkpoint


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) NOT_FOUND: Key batch_normalization/beta/Adam_10 not found in checkpoint
	 [[node save_6/RestoreV2
 (defined at /var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:92)
]]
	 [[save_6/RestoreV2/_17]]
  (1) NOT_FOUND: Key batch_normalization/beta/Adam_10 not found in checkpoint
	 [[node save_6/RestoreV2
 (defined at /var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_60583/1549856815.py:92)
]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node save_6/RestoreV2:
In[0] save_6/Const:	
In[1] save_6/RestoreV2/tensor_names:	
In[2] save_6/RestoreV2/shape_and_slices:

Operation defined at: (most re

In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
weight_decay=1e-4
input = Input((4, 8, 8))
x = input
x = Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(4, 8, 8))(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
x = BatchNormalization()(x)
print(x.shape, input.shape)
p = Conv2D(2, (1,1), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
p = BatchNormalization()(p)
p = Flatten()(p)
p = Dense(8 * 8, activation='softmax')(p)
print(p.shape)
v = Conv2D(2, (1,1), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
v = BatchNormalization()(v)
v = Flatten()(v)
v = Dense(8 * 8, activation='relu')(v)
v = Dense(1, activation='tanh')(v)
print(v.shape)
model = Model(inputs=input, outputs=[p, v])
model.compile(optimizer=Adam(), loss=['categorical_crossentropy', 'mean_squared_error'])
        
